In [18]:
import pandas as pd

In [19]:
df = pd.read_csv(r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\extracted_authors.csv")

In [20]:
df

,author_key,author_name,full_key
0,OL10000073A,Jule Windgeflüster,/authors/OL10000073A
1,OL10000596A,Beth Wildman,/authors/OL10000596A
2,OL10000760A,Jaume Piquet Alcàzar,/authors/OL10000760A
3,OL10001743A,P. M. Gagey,/authors/OL10001743A
4,OL1000176A,Suʻād Rau̓̄f Shīr Muḥammad,/authors/OL1000176A
...,...,...,...
14476246,OL9998741A,Markus Lemke,/authors/OL9998741A
14476247,OL9998762A,Alfred J. Preis,/authors/OL9998762A
14476248,OL999881A,Samīḥ Masʻūd Barqāwī,/authors/OL999881A
14476249,OL9998939A,Digitalia,/authors/OL9998939A


In [4]:
# Calculer le pourcentage de valeurs non-nulles pour chaque colonne
taux_remplissage = (df.count() / len(df) * 100).round(2)

# Afficher les résultats
print("Taux de remplissage par colonne :")
for colonne, taux in taux_remplissage.items():
    print(f"{colonne}: {taux}%")


Taux de remplissage par colonne :
author_key: 100.0%
author_name: 100.0%
full_key: 100.0%


In [23]:
df[df['author_key'] == 'OL14051249A']

,author_key,author_name,full_key
300003,OL14051249A,NaN,/authors/OL14051249A


In [26]:
# Afficher les lignes où author_name est la chaîne 'NaN'
df = df.dropna(subset=['author_name'])

In [27]:
df[df['author_name'].isna()]

,author_key,author_name,full_key


In [28]:
df

,author_key,author_name,full_key
0,OL10000073A,Jule Windgeflüster,/authors/OL10000073A
1,OL10000596A,Beth Wildman,/authors/OL10000596A
2,OL10000760A,Jaume Piquet Alcàzar,/authors/OL10000760A
3,OL10001743A,P. M. Gagey,/authors/OL10001743A
4,OL1000176A,Suʻād Rau̓̄f Shīr Muḥammad,/authors/OL1000176A
...,...,...,...
14476246,OL9998741A,Markus Lemke,/authors/OL9998741A
14476247,OL9998762A,Alfred J. Preis,/authors/OL9998762A
14476248,OL999881A,Samīḥ Masʻūd Barqāwī,/authors/OL999881A
14476249,OL9998939A,Digitalia,/authors/OL9998939A


## TXT editions complete

In [8]:
import gzip
import os
import time
import struct

fichier = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_cdump_2025-05-31.txt.gz"

# MÉTHODE 1 : Lire la taille dans l'en-tête gzip (plus fiable)
def get_gzip_uncompressed_size(filename):
    """Lit la taille décompressée directement depuis l'en-tête gzip"""
    try:
        with open(filename, 'rb') as f:
            # Aller à la fin du fichier pour lire la taille décompressée
            f.seek(-4, 2)  # 4 derniers octets
            return struct.unpack('<I', f.read(4))[0]
    except:
        return None

# MÉTHODE 2 : Estimation plus précise par échantillonnage
def estimate_compression_ratio(filename, num_samples=5, sample_size_mb=5):
    """Prend plusieurs échantillons à différents endroits du fichier"""
    compressed_size = os.path.getsize(filename)
    
    print(f"📁 Taille compressée : {compressed_size:,} octets ({compressed_size / (1024**3):.2f} GB)")
    print(f"🔍 Analyse de {num_samples} échantillons de {sample_size_mb} MB chacun...")
    
    ratios = []
    sample_bytes = sample_size_mb * 1024 * 1024
    
    with open(filename, 'rb') as raw_file:
        file_size = compressed_size
        
        for i in range(num_samples):
            # Prendre des échantillons à différentes positions
            position = (file_size // num_samples) * i
            if position + 1024*1024 >= file_size:  # Éviter la fin du fichier
                continue
                
            raw_file.seek(position)
            compressed_chunk = raw_file.read(1024*1024)  # 1MB d'data compressée
            
            # Décompresser ce chunk pour voir le ratio
            try:
                with gzip.open(filename, 'rb') as gz_file:
                    gz_file.seek(position)  # Cette méthode n'est pas parfaite
                    decompressed_chunk = gz_file.read(sample_bytes)
                    if decompressed_chunk:
                        ratio = len(decompressed_chunk) / len(compressed_chunk)
                        ratios.append(ratio)
                        print(f"  Échantillon {i+1}: ratio ~{ratio:.1f}x")
            except:
                continue
    
    if ratios:
        avg_ratio = sum(ratios) / len(ratios)
        estimated_size = compressed_size * avg_ratio
        return estimated_size, avg_ratio
    return None, None

# MÉTHODE 3 : Analyse simple et rapide
def quick_analysis(filename):
    """Analyse rapide du début du fichier"""
    print("🔍 ANALYSE RAPIDE DU DÉBUT DU FICHIER")
    
    # Analyser les premières lignes
    print("👀 Aperçu du contenu :")
    print("-" * 40)
    
    line_count = 0
    char_count = 0
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 3:
                break
            print(f"{i+1}: {line.strip()[:100]}...")
            line_count += 1
            char_count += len(line)
    
    print("-" * 40)
    print(f"📊 Analyse des premières lignes :")
    print(f"   • Nombre de lignes analysées : {line_count}")
    print(f"   • Caractères par ligne (moyenne) : {char_count // line_count if line_count > 0 else 0}")
    
    # Estimer basé sur la structure typique d'OpenLibrary
    print(f"   • Format détecté : JSON par ligne (JSONL)")
    print(f"   • Type de données : Base de données OpenLibrary")

print("🚀 ANALYSE COMPLÈTE DU FICHIER GZIP")
print("=" * 60)

# Méthode 1 : Taille dans l'en-tête
print("📋 MÉTHODE 1 : Lecture de l'en-tête gzip")
gzip_size = get_gzip_uncompressed_size(fichier)
if gzip_size:
    print(f"✅ Taille décompressée (en-tête gzip) : {gzip_size:,} octets ({gzip_size / (1024**3):.1f} GB)")
else:
    print("❌ Impossible de lire l'en-tête gzip (fichier > 4GB probablement)")

print("\n" + "=" * 60)

# Méthode 2 : Analyse du contenu
print("📋 MÉTHODE 2 : Analyse du contenu")
quick_analysis(fichier)

print("\n" + "=" * 60)

# Estimations basées sur le type de fichier
compressed_size = os.path.getsize(fichier)
print("📋 MÉTHODE 3 : Estimations basées sur le type de données")
print("Les fichiers OpenLibrary dump ont typiquement :")
print("   • Ratio de compression gzip : 4-8x pour du JSON")
print("   • Estimation basse (4x) :", f"{compressed_size * 4 / (1024**3):.1f} GB")
print("   • Estimation haute (8x) :", f"{compressed_size * 8 / (1024**3):.1f} GB")
print("   • Estimation moyenne (6x) :", f"{compressed_size * 6 / (1024**3):.1f} GB")

print(f"\n💡 La taille décompressée est probablement entre {compressed_size * 4 / (1024**3):.0f}-{compressed_size * 8 / (1024**3):.0f} GB")

print("\n" + "=" * 60)

# ESTIMATION DU NOMBRE DE LIGNES
print("📋 MÉTHODE 4 : Estimation du nombre de lignes")

def estimate_line_count(filename, sample_lines=1000):
    """Estime le nombre total de lignes dans le fichier"""
    print(f"🔢 Analyse de {sample_lines} lignes pour estimer le nombre total...")
    
    line_lengths = []
    lines_counted = 0
    total_chars = 0
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= sample_lines:
                break
            line_length = len(line)
            line_lengths.append(line_length)
            total_chars += line_length
            lines_counted += 1
            
            # Afficher le progrès tous les 200 lignes
            if (i + 1) % 200 == 0:
                print(f"   Analysé {i + 1} lignes...")
    
    if lines_counted > 0:
        avg_line_length = total_chars / lines_counted
        min_line_length = min(line_lengths)
        max_line_length = max(line_lengths)
        
        print(f"📊 Statistiques sur {lines_counted} lignes analysées :")
        print(f"   • Longueur moyenne par ligne : {avg_line_length:.0f} caractères")
        print(f"   • Longueur min : {min_line_length} caractères")
        print(f"   • Longueur max : {max_line_length} caractères")
        
        # Estimation basée sur la taille réelle de l'en-tête gzip
        if gzip_size:
            estimated_lines_header = gzip_size / avg_line_length
            print(f"🎯 Estimation basée sur l'en-tête gzip ({gzip_size / (1024**3):.1f} GB) :")
            print(f"   • Nombre de lignes estimé : {estimated_lines_header:,.0f}")
            print(f"   • Soit environ {estimated_lines_header / 1_000_000:.1f} millions de lignes")
        
        # Estimation basée sur les tailles théoriques
        print(f"🎯 Estimations basées sur les tailles théoriques :")
        for ratio, label in [(4, "basse"), (6, "moyenne"), (8, "haute")]:
            theoretical_size = compressed_size * ratio
            estimated_lines = theoretical_size / avg_line_length
            print(f"   • Estimation {label} ({ratio}x, {theoretical_size / (1024**3):.0f} GB) : {estimated_lines:,.0f} lignes ({estimated_lines / 1_000_000:.0f}M)")
        
        return avg_line_length, estimated_lines_header if gzip_size else None
    
    return None, None

# Comptage rapide des lignes sur un échantillon
def quick_line_count_sample(filename, chunk_size_mb=50):
    """Compte les lignes sur un échantillon pour vérifier l'estimation"""
    print(f"🔍 Comptage réel sur un échantillon de {chunk_size_mb} MB...")
    
    chunk_size = chunk_size_mb * 1024 * 1024
    lines_in_sample = 0
    bytes_read = 0
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        while bytes_read < chunk_size:
            line = f.readline()
            if not line:
                break
            lines_in_sample += 1
            bytes_read += len(line.encode('utf-8'))
    
    if bytes_read > 0:
        lines_per_mb = lines_in_sample / (bytes_read / (1024 * 1024))
        print(f"📊 Résultats du comptage réel :")
        print(f"   • Lignes dans {bytes_read / (1024 * 1024):.1f} MB : {lines_in_sample:,}")
        print(f"   • Lignes par MB : {lines_per_mb:,.0f}")
        
        # Projection sur la taille totale
        if gzip_size:
            total_lines_projection = lines_per_mb * (gzip_size / (1024 * 1024))
            print(f"   • Projection sur {gzip_size / (1024**3):.1f} GB : {total_lines_projection:,.0f} lignes")
            print(f"   • Soit environ {total_lines_projection / 1_000_000:.1f} millions de lignes")
        
        return lines_per_mb
    
    return None

# Exécuter les estimations
try:
    avg_length, estimated_lines = estimate_line_count(fichier, sample_lines=500)
    print("\n" + "-" * 50)
    lines_per_mb = quick_line_count_sample(fichier, chunk_size_mb=20)
    
    print(f"\n🎯 RÉSUMÉ DES ESTIMATIONS :")
    if gzip_size:
        print(f"   📁 Taille réelle (en-tête) : {gzip_size / (1024**3):.1f} GB")
        if estimated_lines:
            print(f"   📊 Nombre de lignes estimé : {estimated_lines:,.0f}")
            print(f"   📈 Soit environ {estimated_lines / 1_000_000:.1f} millions de lignes")
    else:
        print(f"   📁 Taille estimée : 170-340 GB")
        print(f"   📊 Nombre de lignes estimé : 170-340 millions de lignes")
        
except Exception as e:
    print(f"❌ Erreur lors de l'estimation des lignes : {e}")

print("\n" + "=" * 60)

# ANALYSE DES TYPES DE LIGNES
print("📋 MÉTHODE 5 : Analyse des types de données")

def analyze_line_types(filename, sample_lines=5000):
    """Analyse les différents types de lignes présents dans le fichier"""
    print(f"🔍 Analyse de {sample_lines} lignes pour identifier les types...")
    
    type_counts = {}
    total_lines = 0
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= sample_lines:
                break
            
            # Extraire le type de ligne (premier champ avant le premier \t)
            parts = line.split('\t', 1)
            if len(parts) > 0:
                line_type = parts[0].strip()
                type_counts[line_type] = type_counts.get(line_type, 0) + 1
                total_lines += 1
            
            # Afficher le progrès tous les 1000 lignes
            if (i + 1) % 1000 == 0:
                print(f"   Analysé {i + 1} lignes...")
    
    if total_lines > 0:
        print(f"\n📊 Types de lignes trouvés dans {total_lines} lignes analysées :")
        print("-" * 50)
        
        # Trier par fréquence (plus fréquent en premier)
        sorted_types = sorted(type_counts.items(), key=lambda x: x[1], reverse=True)
        
        for line_type, count in sorted_types:
            percentage = (count / total_lines) * 100
            print(f"   • {line_type:<25} : {count:>6} lignes ({percentage:>5.1f}%)")
        
        print("-" * 50)
        print(f"📈 Projections sur le fichier complet :")
        
        if gzip_size and estimated_lines:
            print(f"   Basé sur {estimated_lines:,.0f} lignes estimées :")
            for line_type, count in sorted_types[:10]:  # Top 10 seulement
                percentage = (count / total_lines) * 100
                projected_count = (percentage / 100) * estimated_lines
                print(f"   • {line_type:<25} : {projected_count:>10,.0f} lignes estimées")
        
        return type_counts, total_lines
    
    return None, 0

def detailed_type_analysis(filename, sample_lines=2000):
    """Analyse plus détaillée des types avec exemples"""
    print(f"\n🔍 Analyse détaillée avec exemples (sur {sample_lines} lignes)...")
    
    type_examples = {}
    type_structures = {}
    
    with gzip.open(filename, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= sample_lines:
                break
            
            parts = line.split('\t')
            if len(parts) >= 2:
                line_type = parts[0].strip()
                
                # Stocker un exemple pour chaque type (seulement le premier)
                if line_type not in type_examples:
                    type_examples[line_type] = {
                        'example': line.strip()[:200] + "..." if len(line) > 200 else line.strip(),
                        'fields_count': len(parts),
                        'second_field_example': parts[1][:50] + "..." if len(parts[1]) > 50 else parts[1] if len(parts) > 1 else ""
                    }
                
                # Compter la structure (nombre de champs)
                if line_type not in type_structures:
                    type_structures[line_type] = []
                type_structures[line_type].append(len(parts))
    
    print(f"\n📋 DÉTAIL DES TYPES DE DONNÉES :")
    print("=" * 70)
    
    for line_type in sorted(type_examples.keys()):
        example_data = type_examples[line_type]
        field_counts = type_structures[line_type]
        avg_fields = sum(field_counts) / len(field_counts)
        
        print(f"\n🔹 {line_type}")
        print(f"   Nombre moyen de champs : {avg_fields:.1f}")
        print(f"   Exemple de 2ème champ : {example_data['second_field_example']}")
        print(f"   Exemple complet : {example_data['example']}")
        print("   " + "-" * 60)
    
    return type_examples

# Exécuter l'analyse des types
try:
    print("🚀 DÉBUT DE L'ANALYSE DES TYPES")
    type_counts, total_analyzed = analyze_line_types(fichier, sample_lines=3000)
    
    if type_counts:
        print(f"\n📊 {len(type_counts)} types différents trouvés !")
        
        # Analyse détaillée
        type_examples = detailed_type_analysis(fichier, sample_lines=1500)
        
        print(f"\n🎯 RÉSUMÉ DE L'ANALYSE DES TYPES :")
        print(f"   📁 Échantillon analysé : {total_analyzed} lignes")
        print(f"   🔢 Types différents trouvés : {len(type_counts)}")
        print(f"   📊 Type le plus fréquent : {max(type_counts.items(), key=lambda x: x[1])[0]}")
        print(f"   📈 Diversité des données : Très variée (authors, editions, works, etc.)")

except Exception as e:
    print(f"❌ Erreur lors de l'analyse des types : {e}")


🚀 ANALYSE COMPLÈTE DU FICHIER GZIP
📋 MÉTHODE 1 : Lecture de l'en-tête gzip
✅ Taille décompressée (en-tête gzip) : 1,226,586,112 octets (1.1 GB)

📋 MÉTHODE 2 : Analyse du contenu
🔍 ANALYSE RAPIDE DU DÉBUT DU FICHIER
👀 Aperçu du contenu :
----------------------------------------
1: /type/author	/authors/OL5345273A	1	2008-10-05T09:14:51.676778	{"name": "TOM.KIYIHN", "last_modified"...
2: /type/author	/authors/OL5345271A	1	2008-10-04T15:55:36.974594	{"name": "Aksakov, S. T.", "last_modif...
3: /type/author	/authors/OL5345272A	1	2008-10-04T16:04:08.61952	{"name": "Latham,Barbarahttp://webcat.c...
----------------------------------------
📊 Analyse des premières lignes :
   • Nombre de lignes analysées : 3
   • Caractères par ligne (moyenne) : 266
   • Format détecté : JSON par ligne (JSONL)
   • Type de données : Base de données OpenLibrary

📋 MÉTHODE 3 : Estimations basées sur le type de données
Les fichiers OpenLibrary dump ont typiquement :
   • Ratio de compression gzip : 4-8x pour du JS

In [9]:
# ANALYSE DU FICHIER EDITIONS
print("🚀 ANALYSE COMPLÈTE DU FICHIER EDITIONS")
print("=" * 60)

# Chemin du fichier editions (ajustez selon votre structure)
fichier_editions = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_dump_editions_2025-05-31.txt.gz"

# Vérifier si le fichier existe
import os
if not os.path.exists(fichier_editions):
    # Essayer d'autres noms possibles
    base_path = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary"
    possible_names = [
        "ol_dump_editions_latest.txt.gz",
        "editions.txt.gz", 
        "ol_dump_editions.txt.gz",
        "ol_cdump_editions_2025-05-31.txt.gz"
    ]
    
    fichier_editions = None
    for name in possible_names:
        test_path = os.path.join(base_path, name)
        if os.path.exists(test_path):
            fichier_editions = test_path
            break
    
    if fichier_editions is None:
        print("❌ Fichier editions non trouvé. Fichiers disponibles :")
        try:
            for f in os.listdir(base_path):
                if f.endswith('.gz'):
                    print(f"   • {f}")
        except:
            print("   Impossible de lister les fichiers")
        fichier_editions = input("Veuillez entrer le nom exact du fichier editions : ")
        if not os.path.isabs(fichier_editions):
            fichier_editions = os.path.join(base_path, fichier_editions)

if os.path.exists(fichier_editions):
    print(f"📁 Fichier analysé : {os.path.basename(fichier_editions)}")
    
    # MÉTHODE 1 : Taille dans l'en-tête
    print("📋 MÉTHODE 1 : Lecture de l'en-tête gzip")
    gzip_size_editions = get_gzip_uncompressed_size(fichier_editions)
    if gzip_size_editions:
        print(f"✅ Taille décompressée (en-tête gzip) : {gzip_size_editions:,} octets ({gzip_size_editions / (1024**3):.1f} GB)")
    else:
        print("❌ Impossible de lire l'en-tête gzip (fichier > 4GB probablement)")

    print("\n" + "=" * 60)

    # MÉTHODE 2 : Analyse du contenu
    print("📋 MÉTHODE 2 : Analyse du contenu EDITIONS")
    try:
        print("🔍 ANALYSE RAPIDE DU DÉBUT DU FICHIER EDITIONS")
        print("👀 Aperçu du contenu :")
        print("-" * 40)
        
        line_count = 0
        char_count = 0
        
        with gzip.open(fichier_editions, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 5:
                    break
                print(f"{i+1}: {line.strip()[:120]}...")
                line_count += 1
                char_count += len(line)
        
        print("-" * 40)
        print(f"📊 Analyse des premières lignes EDITIONS :")
        print(f"   • Nombre de lignes analysées : {line_count}")
        print(f"   • Caractères par ligne (moyenne) : {char_count // line_count if line_count > 0 else 0}")
        print(f"   • Format détecté : JSON par ligne (JSONL)")
        print(f"   • Type de données : Éditions OpenLibrary")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse du contenu : {e}")

    print("\n" + "=" * 60)

    # MÉTHODE 3 : Estimations basées sur le type de fichier
    compressed_size_editions = os.path.getsize(fichier_editions)
    print("📋 MÉTHODE 3 : Estimations basées sur le type de données EDITIONS")
    print("Les fichiers OpenLibrary editions ont typiquement :")
    print("   • Ratio de compression gzip : 5-10x pour du JSON riche")
    print("   • Estimation basse (5x) :", f"{compressed_size_editions * 5 / (1024**3):.1f} GB")
    print("   • Estimation haute (10x) :", f"{compressed_size_editions * 10 / (1024**3):.1f} GB")
    print("   • Estimation moyenne (7x) :", f"{compressed_size_editions * 7 / (1024**3):.1f} GB")

    print(f"\n💡 La taille décompressée est probablement entre {compressed_size_editions * 5 / (1024**3):.0f}-{compressed_size_editions * 10 / (1024**3):.0f} GB")

    print("\n" + "=" * 60)

    # MÉTHODE 4 : Estimation du nombre de lignes EDITIONS
    print("📋 MÉTHODE 4 : Estimation du nombre de lignes EDITIONS")

    try:
        # Analyser les lignes d'éditions
        print("🔢 Analyse de 300 lignes d'éditions pour estimer le nombre total...")
        
        line_lengths_editions = []
        lines_counted_editions = 0
        total_chars_editions = 0
        
        with gzip.open(fichier_editions, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 300:
                    break
                line_length = len(line)
                line_lengths_editions.append(line_length)
                total_chars_editions += line_length
                lines_counted_editions += 1
                
                if (i + 1) % 100 == 0:
                    print(f"   Analysé {i + 1} lignes...")
        
        if lines_counted_editions > 0:
            avg_line_length_editions = total_chars_editions / lines_counted_editions
            min_line_length_editions = min(line_lengths_editions)
            max_line_length_editions = max(line_lengths_editions)
            
            print(f"📊 Statistiques sur {lines_counted_editions} lignes d'éditions analysées :")
            print(f"   • Longueur moyenne par ligne : {avg_line_length_editions:.0f} caractères")
            print(f"   • Longueur min : {min_line_length_editions} caractères")
            print(f"   • Longueur max : {max_line_length_editions} caractères")
            
            # Estimation basée sur la taille réelle
            if gzip_size_editions:
                estimated_lines_editions = gzip_size_editions / avg_line_length_editions
                print(f"🎯 Estimation basée sur l'en-tête gzip ({gzip_size_editions / (1024**3):.1f} GB) :")
                print(f"   • Nombre de lignes estimé : {estimated_lines_editions:,.0f}")
                print(f"   • Soit environ {estimated_lines_editions / 1_000_000:.1f} millions d'éditions")
            
            # Comptage rapide sur échantillon
            print(f"\n🔍 Comptage réel sur un échantillon de 10 MB...")
            
            chunk_size = 10 * 1024 * 1024
            lines_in_sample = 0
            bytes_read = 0
            
            with gzip.open(fichier_editions, 'rt', encoding='utf-8') as f:
                while bytes_read < chunk_size:
                    line = f.readline()
                    if not line:
                        break
                    lines_in_sample += 1
                    bytes_read += len(line.encode('utf-8'))
            
            if bytes_read > 0:
                lines_per_mb_editions = lines_in_sample / (bytes_read / (1024 * 1024))
                print(f"📊 Résultats du comptage réel EDITIONS :")
                print(f"   • Lignes dans {bytes_read / (1024 * 1024):.1f} MB : {lines_in_sample:,}")
                print(f"   • Lignes par MB : {lines_per_mb_editions:,.0f}")
                
                if gzip_size_editions:
                    total_lines_projection_editions = lines_per_mb_editions * (gzip_size_editions / (1024 * 1024))
                    print(f"   • Projection sur {gzip_size_editions / (1024**3):.1f} GB : {total_lines_projection_editions:,.0f} éditions")
                    print(f"   • Soit environ {total_lines_projection_editions / 1_000_000:.1f} millions d'éditions")

    except Exception as e:
        print(f"❌ Erreur lors de l'estimation des lignes : {e}")

    print("\n" + "=" * 60)

    # MÉTHODE 5 : Analyse des types de données EDITIONS
    print("📋 MÉTHODE 5 : Analyse des types de données EDITIONS")

    try:
        print("🚀 DÉBUT DE L'ANALYSE DES TYPES D'ÉDITIONS")
        print(f"🔍 Analyse de 2000 lignes pour identifier les types d'éditions...")
        
        type_counts_editions = {}
        total_lines_editions = 0
        
        with gzip.open(fichier_editions, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 2000:
                    break
                
                parts = line.split('\t', 1)
                if len(parts) > 0:
                    line_type = parts[0].strip()
                    type_counts_editions[line_type] = type_counts_editions.get(line_type, 0) + 1
                    total_lines_editions += 1
                
                if (i + 1) % 500 == 0:
                    print(f"   Analysé {i + 1} lignes...")
        
        if total_lines_editions > 0:
            print(f"\n📊 Types de lignes trouvés dans {total_lines_editions} lignes d'éditions :")
            print("-" * 50)
            
            sorted_types_editions = sorted(type_counts_editions.items(), key=lambda x: x[1], reverse=True)
            
            for line_type, count in sorted_types_editions:
                percentage = (count / total_lines_editions) * 100
                print(f"   • {line_type:<25} : {count:>6} lignes ({percentage:>5.1f}%)")
            
            print("-" * 50)
            
            # Analyse détaillée avec exemples d'éditions
            print(f"\n🔍 Analyse détaillée des éditions avec exemples...")
            
            type_examples_editions = {}
            
            with gzip.open(fichier_editions, 'rt', encoding='utf-8') as f:
                for i, line in enumerate(f):
                    if i >= 1000:
                        break
                    
                    parts = line.split('\t')
                    if len(parts) >= 2:
                        line_type = parts[0].strip()
                        
                        if line_type not in type_examples_editions:
                            type_examples_editions[line_type] = {
                                'example': line.strip()[:200] + "..." if len(line) > 200 else line.strip(),
                                'second_field': parts[1][:50] + "..." if len(parts[1]) > 50 else parts[1]
                            }
            
            print(f"\n📋 DÉTAIL DES TYPES D'ÉDITIONS :")
            print("=" * 70)
            
            for line_type in sorted(type_examples_editions.keys()):
                example_data = type_examples_editions[line_type]
                print(f"\n🔹 {line_type}")
                print(f"   Exemple de 2ème champ : {example_data['second_field']}")
                print(f"   Exemple complet : {example_data['example']}")
                print("   " + "-" * 60)
            
            print(f"\n🎯 RÉSUMÉ DE L'ANALYSE DES ÉDITIONS :")
            print(f"   📁 Échantillon analysé : {total_lines_editions} lignes")
            print(f"   🔢 Types différents trouvés : {len(type_counts_editions)}")
            if type_counts_editions:
                print(f"   📊 Type le plus fréquent : {max(type_counts_editions.items(), key=lambda x: x[1])[0]}")
            
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse des types d'éditions : {e}")

else:
    print("❌ Fichier editions non trouvé !")
    print("Veuillez vérifier le chemin du fichier.")

print("\n" + "🎯" * 30)
print("COMPARAISON FICHIER PRINCIPAL vs ÉDITIONS")
print("🎯" * 30)

try:
    if 'gzip_size' in locals() and gzip_size and 'gzip_size_editions' in locals() and gzip_size_editions:
        print(f"📊 Taille fichier principal : {gzip_size / (1024**3):.1f} GB")
        print(f"📊 Taille fichier éditions : {gzip_size_editions / (1024**3):.1f} GB")
        print(f"📈 Ratio éditions/principal : {gzip_size_editions / gzip_size:.1f}x")
        
        if 'estimated_lines' in locals() and 'estimated_lines_editions' in locals():
            print(f"🔢 Lignes fichier principal : ~{estimated_lines:,.0f}")
            print(f"🔢 Lignes fichier éditions : ~{estimated_lines_editions:,.0f}")
    else:
        print("⚠️ Données incomplètes pour la comparaison")
except Exception as e:
    print(f"❌ Erreur dans la comparaison : {e}")


🚀 ANALYSE COMPLÈTE DU FICHIER EDITIONS
📁 Fichier analysé : ol_dump_editions_2025-05-31.txt.gz
📋 MÉTHODE 1 : Lecture de l'en-tête gzip
✅ Taille décompressée (en-tête gzip) : 1,484,839,911 octets (1.4 GB)

📋 MÉTHODE 2 : Analyse du contenu EDITIONS
🔍 ANALYSE RAPIDE DU DÉBUT DU FICHIER EDITIONS
👀 Aperçu du contenu :
----------------------------------------
1: /type/edition	/books/OL10000079M	2	2010-03-12T00:00:48.298004	{"publishers": ["Stationery Office Books"], "physical_form...
2: /type/edition	/books/OL10000715M	3	2011-04-26T05:54:13.372021	{"publishers": ["Stationery Office Books"], "physical_form...
3: /type/edition	/books/OL10001085M	3	2011-04-30T15:33:17.349849	{"publishers": ["Stationery Office Books"], "physical_form...
4: /type/edition	/books/OL10001190M	3	2011-04-22T23:18:54.840113	{"publishers": ["Stationery Office Books"], "physical_form...
5: /type/edition	/books/OL1000121M	9	2023-01-14T19:33:59.345503	{"publishers": ["American Quilter's Society"], "number_of_p...
----------

## TXT Editions

In [13]:
# EXTRACTION D'OBJETS JSON AUTEURS (FICHIER DÉCOMPRESSÉ)
print("👤 EXTRACTION D'OBJETS JSON AUTEURS")
print("=" * 60)

import json
import os

def extract_authors_json_objects(filename, num_objects=5):
    """Extrait et affiche des objets JSON d'auteurs du fichier décompressé"""
    print(f"📋 Extraction de {num_objects} objets JSON d'auteurs du fichier...")
    
    extracted_objects = []
    objects_found = 0
    lines_processed = 0
    
    try:
        # Lire le fichier texte normal (pas gzip)
        with open(filename, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                lines_processed += 1
                
                if objects_found >= num_objects:
                    break
                
                # Diviser la ligne en champs (séparés par des tabulations)
                parts = line.strip().split('\t')
                if len(parts) >= 5:  # Type, ID, Revision, Timestamp, JSON
                    object_type = parts[0]
                    object_id = parts[1]
                    revision = parts[2]
                    timestamp = parts[3]
                    json_data = parts[4]
                    
                    # Filtrer uniquement les auteurs
                    if object_type != '/type/author':
                        continue
                    
                    try:
                        # Parser le JSON
                        parsed_json = json.loads(json_data)
                        
                        extracted_objects.append({
                            'line_number': i + 1,
                            'type': object_type,
                            'id': object_id,
                            'revision': revision,
                            'timestamp': timestamp,
                            'json_object': parsed_json
                        })
                        
                        objects_found += 1
                        
                    except json.JSONDecodeError as e:
                        print(f"⚠️ Erreur JSON à la ligne {i+1}: {str(e)[:100]}...")
                        continue
                
                # Afficher le progrès tous les 10000 lignes
                if lines_processed % 10000 == 0:
                    print(f"   Traité {lines_processed:,} lignes, trouvé {objects_found} auteurs valides...")
    
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        return []
    
    print(f"✅ Extraction terminée : {objects_found} auteurs extraits sur {lines_processed:,} lignes traitées")
    return extracted_objects

def display_authors_json_objects(objects):
    """Affiche les objets JSON d'auteurs de manière formatée"""
    for i, obj in enumerate(objects):
        print(f"\n{'='*80}")
        print(f"👤 AUTEUR {i+1}")
        print(f"{'='*80}")
        print(f"🔸 Type: {obj['type']}")
        print(f"🔸 ID: {obj['id']}")
        print(f"🔸 Révision: {obj['revision']}")
        print(f"🔸 Timestamp: {obj['timestamp']}")
        print(f"🔸 Ligne dans le fichier: {obj['line_number']}")
        
        # Extraire les infos principales de l'auteur
        json_obj = obj['json_object']
        print(f"\n📋 INFORMATIONS PRINCIPALES:")
        print("-" * 30)
        
        if 'name' in json_obj:
            print(f"📝 Nom: {json_obj['name']}")
        
        if 'birth_date' in json_obj:
            print(f"🎂 Date de naissance: {json_obj['birth_date']}")
        
        if 'death_date' in json_obj:
            print(f"⚰️ Date de décès: {json_obj['death_date']}")
        
        if 'bio' in json_obj:
            bio = json_obj['bio']
            if isinstance(bio, dict) and 'value' in bio:
                bio_text = bio['value'][:200] + "..." if len(bio['value']) > 200 else bio['value']
                print(f"📚 Biographie: {bio_text}")
            elif isinstance(bio, str):
                bio_text = bio[:200] + "..." if len(bio) > 200 else bio
                print(f"📚 Biographie: {bio_text}")
        
        if 'alternate_names' in json_obj and json_obj['alternate_names']:
            print(f"🔄 Noms alternatifs: {', '.join(json_obj['alternate_names'][:3])}")
        
        if 'wikipedia' in json_obj:
            print(f"🌐 Wikipedia: {json_obj['wikipedia']}")
        
        print(f"\n📋 OBJET JSON COMPLET:")
        print("-" * 50)
        
        # Afficher le JSON formaté
        try:
            formatted_json = json.dumps(json_obj, indent=2, ensure_ascii=False)
            # Limiter l'affichage si trop long
            if len(formatted_json) > 1500:
                lines = formatted_json.split('\n')
                displayed_lines = lines[:40]  # Première partie
                print('\n'.join(displayed_lines))
                print(f"\n... [TRONQUÉ - {len(lines)-40} lignes supplémentaires] ...")
                print('\n'.join(lines[-3:]))  # Dernières lignes
            else:
                print(formatted_json)
        except Exception as e:
            print(f"❌ Erreur d'affichage: {e}")
        
        print("-" * 50)

def analyze_authors_structure(objects):
    """Analyse la structure spécialisée des objets auteurs"""
    print(f"\n👤 ANALYSE SPÉCIALISÉE DES AUTEURS")
    print("=" * 50)
    
    # Analyser les champs
    all_keys = set()
    key_frequency = {}
    
    # Statistiques spécialisées
    authors_with_bio = 0
    authors_with_birth_date = 0
    authors_with_death_date = 0
    authors_with_wikipedia = 0
    authors_with_alternate_names = 0
    
    birth_years = []
    death_years = []
    name_lengths = []
    
    for obj in objects:
        json_obj = obj['json_object']
        if isinstance(json_obj, dict):
            for key in json_obj.keys():
                all_keys.add(key)
                key_frequency[key] = key_frequency.get(key, 0) + 1
            
            # Statistiques spécialisées
            if 'bio' in json_obj:
                authors_with_bio += 1
            
            if 'birth_date' in json_obj:
                authors_with_birth_date += 1
                try:
                    birth_date = json_obj['birth_date']
                    if isinstance(birth_date, str) and len(birth_date) >= 4:
                        year = int(birth_date[:4])
                        if 1000 <= year <= 2025:
                            birth_years.append(year)
                except:
                    pass
            
            if 'death_date' in json_obj:
                authors_with_death_date += 1
                try:
                    death_date = json_obj['death_date']
                    if isinstance(death_date, str) and len(death_date) >= 4:
                        year = int(death_date[:4])
                        if 1000 <= year <= 2025:
                            death_years.append(year)
                except:
                    pass
            
            if 'wikipedia' in json_obj:
                authors_with_wikipedia += 1
            
            if 'alternate_names' in json_obj and json_obj['alternate_names']:
                authors_with_alternate_names += 1
            
            if 'name' in json_obj:
                name_lengths.append(len(json_obj['name']))
    
    # Affichage des statistiques
    print(f"📊 CHAMPS TROUVÉS DANS LES OBJETS AUTEURS :")
    sorted_keys = sorted(key_frequency.items(), key=lambda x: x[1], reverse=True)
    
    for key, freq in sorted_keys:
        percentage = (freq / len(objects)) * 100
        print(f"   • {key:<25} : {freq}/{len(objects)} auteurs ({percentage:.0f}%)")
    
    print(f"\n📈 STATISTIQUES SPÉCIALISÉES :")
    print(f"   📚 Auteurs avec biographie    : {authors_with_bio}/{len(objects)} ({(authors_with_bio/len(objects)*100):.0f}%)")
    print(f"   🎂 Auteurs avec date naissance: {authors_with_birth_date}/{len(objects)} ({(authors_with_birth_date/len(objects)*100):.0f}%)")
    print(f"   ⚰️ Auteurs avec date décès    : {authors_with_death_date}/{len(objects)} ({(authors_with_death_date/len(objects)*100):.0f}%)")
    print(f"   🌐 Auteurs avec Wikipedia     : {authors_with_wikipedia}/{len(objects)} ({(authors_with_wikipedia/len(objects)*100):.0f}%)")
    print(f"   🔄 Auteurs avec noms alternatifs: {authors_with_alternate_names}/{len(objects)} ({(authors_with_alternate_names/len(objects)*100):.0f}%)")
    
    if birth_years:
        print(f"\n📅 ANNÉES DE NAISSANCE :")
        print(f"   • Plus ancienne : {min(birth_years)}")
        print(f"   • Plus récente  : {max(birth_years)}")
        print(f"   • Moyenne       : {sum(birth_years)/len(birth_years):.0f}")
    
    if death_years:
        print(f"\n⚰️ ANNÉES DE DÉCÈS :")
        print(f"   • Plus ancienne : {min(death_years)}")
        print(f"   • Plus récente  : {max(death_years)}")
        print(f"   • Moyenne       : {sum(death_years)/len(death_years):.0f}")
    
    if name_lengths:
        print(f"\n📝 LONGUEUR DES NOMS :")
        print(f"   • Plus court : {min(name_lengths)} caractères")
        print(f"   • Plus long  : {max(name_lengths)} caractères")
        print(f"   • Moyenne    : {sum(name_lengths)/len(name_lengths):.1f} caractères")

# RECHERCHE DU FICHIER AUTEURS DÉCOMPRESSÉ
print("🔍 RECHERCHE DU FICHIER AUTEURS DÉCOMPRESSÉ")
print("-" * 60)

# Chemin direct vers le fichier des auteurs décompressé
fichier_auteurs = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_dump_authors_2025-05-31.txt\ol_dump_authors_2025-05-31.txt"

# Vérifier que le fichier existe
if not os.path.exists(fichier_auteurs):
    print(f"❌ Fichier non trouvé : {fichier_auteurs}")
    print("📂 Fichiers .txt disponibles dans le dossier :")
    try:
        for f in os.listdir(base_path):
            if f.endswith('.txt'):
                print(f"   • {f}")
    except:
        print("   Impossible de lister les fichiers")
    fichier_auteurs = None

if fichier_auteurs and os.path.exists(fichier_auteurs):
    print(f"✅ Fichier trouvé : {os.path.basename(fichier_auteurs)}")
    file_size = os.path.getsize(fichier_auteurs)
    print(f"📁 Taille du fichier : {file_size:,} octets ({file_size / (1024**3):.2f} GB)")
    
    # Extraction des objets JSON d'auteurs
    authors_objects = extract_authors_json_objects(fichier_auteurs, num_objects=5)
    
    if authors_objects:
        display_authors_json_objects(authors_objects)
        analyze_authors_structure(authors_objects)
        
        print(f"\n💡 INSIGHTS SUR LES DONNÉES D'AUTEURS :")
        print(f"🔍 Les objets auteurs contiennent :")
        print(f"   • Informations biographiques (nom, dates, bio)")
        print(f"   • Liens externes (Wikipedia, sites web)")
        print(f"   • Métadonnées de révision et timestamps")
        print(f"   • Noms alternatifs et variantes")
        print(f"   • Identifiants uniques OpenLibrary")
    else:
        print("❌ Aucun objet JSON d'auteur valide trouvé")

else:
    print("❌ Fichier auteurs décompressé non accessible")
    print("💡 Assurez-vous que le fichier a été décompressé et est accessible")


👤 EXTRACTION D'OBJETS JSON AUTEURS
🔍 RECHERCHE DU FICHIER AUTEURS DÉCOMPRESSÉ
------------------------------------------------------------
✅ Fichier trouvé : ol_dump_authors_2025-05-31.txt
📁 Taille du fichier : 5,467,491,837 octets (5.09 GB)
📋 Extraction de 5 objets JSON d'auteurs du fichier...
✅ Extraction terminée : 5 auteurs extraits sur 6 lignes traitées

👤 AUTEUR 1
🔸 Type: /type/author
🔸 ID: /authors/OL10000073A
🔸 Révision: 3
🔸 Timestamp: 2024-12-30T07:49:12.172071
🔸 Ligne dans le fichier: 1

📋 INFORMATIONS PRINCIPALES:
------------------------------
📝 Nom: Jule Windgeflüster

📋 OBJET JSON COMPLET:
--------------------------------------------------
{
  "source_records": [
    "bwb:9783850407960"
  ],
  "name": "Jule Windgeflüster",
  "type": {
    "key": "/type/author"
  },
  "key": "/authors/OL10000073A",
  "latest_revision": 3,
  "revision": 3,
  "created": {
    "type": "/type/datetime",
    "value": "2021-12-26T21:23:27.331795"
  },
  "last_modified": {
    "type": "/type/date

In [14]:
# RECHERCHE APPROFONDIE DES MÉTADONNÉES D'AUTEURS
print("🔍 RECHERCHE APPROFONDIE DES MÉTADONNÉES D'AUTEURS")
print("=" * 70)

import json
import os
from collections import defaultdict

def search_author_metadata(filename, sample_size=10000):
    """Recherche spécifiquement les métadonnées riches d'auteurs"""
    print(f"🔎 Analyse de {sample_size:,} auteurs pour trouver les métadonnées...")
    
    # Compteurs pour différents types de métadonnées
    stats = {
        'total_authors': 0,
        'with_bio': 0,
        'with_birth_date': 0,
        'with_death_date': 0,
        'with_wikipedia': 0,
        'with_wikidata': 0,
        'with_alternate_names': 0,
        'with_personal_name': 0,
        'with_photos': 0,
        'with_links': 0,
        'with_website': 0
    }
    
    # Exemples d'auteurs avec métadonnées
    examples = {
        'bio_examples': [],
        'birth_date_examples': [],
        'wikipedia_examples': [],
        'wikidata_examples': [],
        'rich_profiles': []  # Auteurs avec beaucoup de métadonnées
    }
    
    # Types de champs trouvés
    all_fields = defaultdict(int)
    
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if stats['total_authors'] >= sample_size:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5 and parts[0] == '/type/author':
                    try:
                        json_data = json.loads(parts[4])\n                        stats['total_authors'] += 1
                        
                        # Compter tous les champs présents
                        field_count = 0
                        for key in json_data.keys():
                            all_fields[key] += 1
                            field_count += 1
                        
                        # Vérifier la biographie
                        if 'bio' in json_data:
                            stats['with_bio'] += 1
                            bio_text = \"\"\n                            if isinstance(json_data['bio'], dict) and 'value' in json_data['bio']:
                                bio_text = json_data['bio']['value']
                            elif isinstance(json_data['bio'], str):
                                bio_text = json_data['bio']
                            
                            if bio_text and len(examples['bio_examples']) < 3:
                                examples['bio_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'bio': bio_text[:200] + \"...\" if len(bio_text) > 200 else bio_text
                                })
                        
                        # Vérifier les dates de naissance
                        if 'birth_date' in json_data:
                            stats['with_birth_date'] += 1
                            if len(examples['birth_date_examples']) < 3:
                                examples['birth_date_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'birth_date': json_data['birth_date']
                                })
                        
                        # Vérifier les dates de décès
                        if 'death_date' in json_data:
                            stats['with_death_date'] += 1
                        
                        # Vérifier Wikipedia
                        if 'wikipedia' in json_data:
                            stats['with_wikipedia'] += 1
                            if len(examples['wikipedia_examples']) < 3:
                                examples['wikipedia_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'wikipedia': json_data['wikipedia']
                                })
                        
                        # Vérifier Wikidata (peut être dans 'remote_ids' ou directement)
                        wikidata_found = False
                        if 'remote_ids' in json_data:
                            remote_ids = json_data['remote_ids']
                            if isinstance(remote_ids, dict) and 'wikidata' in remote_ids:
                                stats['with_wikidata'] += 1
                                wikidata_found = True
                                if len(examples['wikidata_examples']) < 3:
                                    examples['wikidata_examples'].append({
                                        'name': json_data.get('name', 'N/A'),
                                        'id': parts[1],
                                        'wikidata': remote_ids['wikidata']
                                    })
                        
                        # Autres champs intéressants
                        if 'alternate_names' in json_data and json_data['alternate_names']:
                            stats['with_alternate_names'] += 1
                        
                        if 'personal_name' in json_data:
                            stats['with_personal_name'] += 1
                        
                        if 'photos' in json_data:
                            stats['with_photos'] += 1
                        
                        if 'links' in json_data:
                            stats['with_links'] += 1
                        
                        if 'website' in json_data:
                            stats['with_website'] += 1
                        
                        # Identifier les profils riches (avec beaucoup de métadonnées)
                        if field_count >= 8 and len(examples['rich_profiles']) < 5:
                            examples['rich_profiles'].append({
                                'name': json_data.get('name', 'N/A'),
                                'id': parts[1],
                                'field_count': field_count,
                                'fields': list(json_data.keys()),
                                'full_object': json_data
                            })
                        
                        # Afficher le progrès
                        if stats['total_authors'] % 1000 == 0:
                            print(f\"   Analysé {stats['total_authors']:,} auteurs... (Bio: {stats['with_bio']}, Dates: {stats['with_birth_date']}, Wikipedia: {stats['with_wikipedia']})\")\n                    
                    except json.JSONDecodeError:
                        continue
    
    except Exception as e:
        print(f\"❌ Erreur lors de l'analyse : {e}\")\n        return None, None
    
    return stats, examples, all_fields

def display_metadata_analysis(stats, examples, all_fields):
    \"\"\"Affiche les résultats de l'analyse des métadonnées\"\"\"
    print(f\"\\n📊 RÉSULTATS DE L'ANALYSE DES MÉTADONNÉES\")
    print(\"=\" * 60)
    
    total = stats['total_authors']
    print(f\"👥 Total d'auteurs analysés : {total:,}\")\n    print(f\"\\n📈 PRÉSENCE DES MÉTADONNÉES :\")\n    print(f\"   📚 Auteurs avec biographie     : {stats['with_bio']:,} ({stats['with_bio']/total*100:.1f}%)\")\n    print(f\"   🎂 Auteurs avec date naissance : {stats['with_birth_date']:,} ({stats['with_birth_date']/total*100:.1f}%)\")\n    print(f\"   ⚰️ Auteurs avec date décès     : {stats['with_death_date']:,} ({stats['with_death_date']/total*100:.1f}%)\")\n    print(f\"   🌐 Auteurs avec Wikipedia      : {stats['with_wikipedia']:,} ({stats['with_wikipedia']/total*100:.1f}%)\")\n    print(f\"   🔗 Auteurs avec Wikidata       : {stats['with_wikidata']:,} ({stats['with_wikidata']/total*100:.1f}%)\")\n    print(f\"   🔄 Auteurs avec noms alternatifs: {stats['with_alternate_names']:,} ({stats['with_alternate_names']/total*100:.1f}%)\")\n    print(f\"   📝 Auteurs avec nom personnel  : {stats['with_personal_name']:,} ({stats['with_personal_name']/total*100:.1f}%)\")\n    print(f\"   📸 Auteurs avec photos         : {stats['with_photos']:,} ({stats['with_photos']/total*100:.1f}%)\")\n    print(f\"   🔗 Auteurs avec liens externes : {stats['with_links']:,} ({stats['with_links']/total*100:.1f}%)\")\n    print(f\"   🌐 Auteurs avec site web       : {stats['with_website']:,} ({stats['with_website']/total*100:.1f}%)\")\n    \n    print(f\"\\n📋 CHAMPS LES PLUS FRÉQUENTS :\")\n    sorted_fields = sorted(all_fields.items(), key=lambda x: x[1], reverse=True)\n    for field, count in sorted_fields[:15]:\n        percentage = (count / total) * 100\n        print(f\"   • {field:<25} : {count:>6,} auteurs ({percentage:>5.1f}%)\")\n    \n    # Afficher des exemples\n    if examples['bio_examples']:\n        print(f\"\\n📚 EXEMPLES D'AUTEURS AVEC BIOGRAPHIES :\")\n        for i, ex in enumerate(examples['bio_examples'], 1):\n            print(f\"   {i}. {ex['name']} ({ex['id']})\")\n            print(f\"      Bio: {ex['bio']}\")\n    \n    if examples['birth_date_examples']:\n        print(f\"\\n🎂 EXEMPLES D'AUTEURS AVEC DATES DE NAISSANCE :\")\n        for i, ex in enumerate(examples['birth_date_examples'], 1):\n            print(f\"   {i}. {ex['name']} ({ex['id']})\")\n            print(f\"      Né(e) le: {ex['birth_date']}\")\n    \n    if examples['wikipedia_examples']:\n        print(f\"\\n🌐 EXEMPLES D'AUTEURS AVEC WIKIPEDIA :\")\n        for i, ex in enumerate(examples['wikipedia_examples'], 1):\n            print(f\"   {i}. {ex['name']} ({ex['id']})\")\n            print(f\"      Wikipedia: {ex['wikipedia']}\")\n    \n    if examples['wikidata_examples']:\n        print(f\"\\n🔗 EXEMPLES D'AUTEURS AVEC WIKIDATA :\")\n        for i, ex in enumerate(examples['wikidata_examples'], 1):\n            print(f\"   {i}. {ex['name']} ({ex['id']})\")\n            print(f\"      Wikidata: {ex['wikidata']}\")\n    \n    if examples['rich_profiles']:\n        print(f\"\\n⭐ EXEMPLES DE PROFILS RICHES (beaucoup de métadonnées) :\")\n        for i, ex in enumerate(examples['rich_profiles'], 1):\n            print(f\"   {i}. {ex['name']} ({ex['id']}) - {ex['field_count']} champs\")\n            print(f\"      Champs: {', '.join(ex['fields'][:8])}{'...' if len(ex['fields']) > 8 else ''}\")\n\ndef display_rich_profile_details(examples):\n    \"\"\"Affiche les détails complets d'un profil riche\"\"\"\n    if examples['rich_profiles']:\n        print(f\"\\n🔍 DÉTAIL D'UN PROFIL RICHE COMPLET :\")\n        print(\"=\" * 60)\n        \n        rich_author = examples['rich_profiles'][0]  # Premier profil riche\n        print(f\"👤 Auteur: {rich_author['name']}\")\n        print(f\"🆔 ID: {rich_author['id']}\")\n        print(f\"📊 Nombre de champs: {rich_author['field_count']}\")\n        \n        print(f\"\\n📋 OBJET JSON COMPLET :\")\n        print(\"-\" * 50)\n        try:\n            formatted_json = json.dumps(rich_author['full_object'], indent=2, ensure_ascii=False)\n            if len(formatted_json) > 2000:\n                lines = formatted_json.split('\\n')\n                print('\\n'.join(lines[:60]))\n                print(f\"\\n... [TRONQUÉ - {len(lines)-60} lignes supplémentaires] ...\")\n                print('\\n'.join(lines[-5:]))\n            else:\n                print(formatted_json)\n        except:\n            print(\"❌ Erreur d'affichage du JSON\")\n        print(\"-\" * 50)

# EXÉCUTION DE L'ANALYSE
fichier_auteurs = r\"C:\\Users\\dd758\\Formation_IA_Greta\\Projet_possible certif\\Livre_analyse\\data_book\\databook\\data\\fichier_openlibrary\\ol_dump_authors_2025-05-31.txt\\ol_dump_authors_2025-05-31.txt\"

if os.path.exists(fichier_auteurs):
    print(f\"✅ Fichier trouvé : {os.path.basename(fichier_auteurs)}\")\n    print(f\"📁 Analyse en cours sur un échantillon de 50,000 auteurs...\")\n    \n    # Analyser un échantillon plus large\n    stats, examples, all_fields = search_author_metadata(fichier_auteurs, sample_size=50000)\n    \n    if stats:\n        display_metadata_analysis(stats, examples, all_fields)\n        display_rich_profile_details(examples)\n        \n        print(f\"\\n💡 CONCLUSIONS SUR LES MÉTADONNÉES D'AUTEURS :\")\n        print(f\"\" * 60)\n        \n        bio_rate = stats['with_bio'] / stats['total_authors'] * 100\n        date_rate = stats['with_birth_date'] / stats['total_authors'] * 100\n        wiki_rate = stats['with_wikipedia'] / stats['total_authors'] * 100\n        wikidata_rate = stats['with_wikidata'] / stats['total_authors'] * 100\n        \n        if bio_rate > 10:\n            print(f\"✅ EXCELLENT : {bio_rate:.1f}% des auteurs ont une biographie\")\n        elif bio_rate > 5:\n            print(f\"✅ BON : {bio_rate:.1f}% des auteurs ont une biographie\")\n        else:\n            print(f\"⚠️ LIMITÉ : Seulement {bio_rate:.1f}% des auteurs ont une biographie\")\n        \n        if date_rate > 20:\n            print(f\"✅ EXCELLENT : {date_rate:.1f}% des auteurs ont une date de naissance\")\n        elif date_rate > 10:\n            print(f\"✅ BON : {date_rate:.1f}% des auteurs ont une date de naissance\")\n        else:\n            print(f\"⚠️ LIMITÉ : Seulement {date_rate:.1f}% des auteurs ont une date de naissance\")\n        \n        total_external = wiki_rate + wikidata_rate\n        if total_external > 30:\n            print(f\"✅ EXCELLENT : {total_external:.1f}% des auteurs ont des liens externes (Wikipedia/Wikidata)\")\n        elif total_external > 15:\n            print(f\"✅ BON : {total_external:.1f}% des auteurs ont des liens externes\")\n        else:\n            print(f\"⚠️ LIMITÉ : Seulement {total_external:.1f}% des auteurs ont des liens externes\")\n        \n        print(f\"\\n🎯 RECOMMANDATIONS :\")\n        rich_rate = len(examples['rich_profiles']) / stats['total_authors'] * 100\n        if rich_rate > 0.1:\n            print(f\"💡 Il existe des profils riches ({rich_rate:.2f}%) - concentrez-vous sur ceux-ci pour l'analyse\")\n        \n        if bio_rate > 5:\n            print(f\"📚 Les biographies sont présentes - utilisez-les pour l'analyse de contenu\")\n        \n        if date_rate > 10:\n            print(f\"📅 Les dates sont disponibles - possibilité d'analyse temporelle\")\n        \n        if wiki_rate > 5 or wikidata_rate > 5:\n            print(f\"🔗 Liens externes disponibles - possibilité d'enrichissement des données\")\n        \n    else:\n        print(\"❌ Échec de l'analyse des métadonnées\")\nelse:\n    print(\"❌ Fichier d'auteurs non trouvé\")\n    print(f\"Chemin recherché : {fichier_auteurs}\")


SyntaxError: unexpected character after line continuation character (1698817028.py, line 49)

## TXT Auteur

In [16]:
# RECHERCHE APPROFONDIE DES MÉTADONNÉES D'AUTEURS
print("🔍 RECHERCHE APPROFONDIE DES MÉTADONNÉES D'AUTEURS")
print("=" * 70)

import json
import os
from collections import defaultdict

def search_author_metadata(filename, sample_size=10000):
    """Recherche spécifiquement les métadonnées riches d'auteurs"""
    print(f"🔎 Analyse de {sample_size:,} auteurs pour trouver les métadonnées...")
    
    # Compteurs pour différents types de métadonnées
    stats = {
        'total_authors': 0,
        'with_bio': 0,
        'with_birth_date': 0,
        'with_death_date': 0,
        'with_wikipedia': 0,
        'with_wikidata': 0,
        'with_alternate_names': 0,
        'with_personal_name': 0,
        'with_photos': 0,
        'with_links': 0,
        'with_website': 0
    }
    
    # Exemples d'auteurs avec métadonnées
    examples = {
        'bio_examples': [],
        'birth_date_examples': [],
        'wikipedia_examples': [],
        'wikidata_examples': [],
        'rich_profiles': []  # Auteurs avec beaucoup de métadonnées
    }
    
    # Types de champs trouvés
    all_fields = defaultdict(int)
    
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if stats['total_authors'] >= sample_size:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5 and parts[0] == '/type/author':
                    try:
                        json_data = json.loads(parts[4])
                        stats['total_authors'] += 1
                        
                        # Compter tous les champs présents
                        field_count = 0
                        for key in json_data.keys():
                            all_fields[key] += 1
                            field_count += 1
                        
                        # Vérifier la biographie
                        if 'bio' in json_data:
                            stats['with_bio'] += 1
                            bio_text = ""
                            if isinstance(json_data['bio'], dict) and 'value' in json_data['bio']:
                                bio_text = json_data['bio']['value']
                            elif isinstance(json_data['bio'], str):
                                bio_text = json_data['bio']
                            
                            if bio_text and len(examples['bio_examples']) < 3:
                                examples['bio_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'bio': bio_text[:200] + "..." if len(bio_text) > 200 else bio_text
                                })
                        
                        # Vérifier les dates de naissance
                        if 'birth_date' in json_data:
                            stats['with_birth_date'] += 1
                            if len(examples['birth_date_examples']) < 3:
                                examples['birth_date_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'birth_date': json_data['birth_date']
                                })
                        
                        # Vérifier les dates de décès
                        if 'death_date' in json_data:
                            stats['with_death_date'] += 1
                        
                        # Vérifier Wikipedia
                        if 'wikipedia' in json_data:
                            stats['with_wikipedia'] += 1
                            if len(examples['wikipedia_examples']) < 3:
                                examples['wikipedia_examples'].append({
                                    'name': json_data.get('name', 'N/A'),
                                    'id': parts[1],
                                    'wikipedia': json_data['wikipedia']
                                })
                        
                        # Vérifier Wikidata (peut être dans 'remote_ids' ou directement)
                        wikidata_found = False
                        if 'remote_ids' in json_data:
                            remote_ids = json_data['remote_ids']
                            if isinstance(remote_ids, dict) and 'wikidata' in remote_ids:
                                stats['with_wikidata'] += 1
                                wikidata_found = True
                                if len(examples['wikidata_examples']) < 3:
                                    examples['wikidata_examples'].append({
                                        'name': json_data.get('name', 'N/A'),
                                        'id': parts[1],
                                        'wikidata': remote_ids['wikidata']
                                    })
                        
                        # Autres champs intéressants
                        if 'alternate_names' in json_data and json_data['alternate_names']:
                            stats['with_alternate_names'] += 1
                        
                        if 'personal_name' in json_data:
                            stats['with_personal_name'] += 1
                        
                        if 'photos' in json_data:
                            stats['with_photos'] += 1
                        
                        if 'links' in json_data:
                            stats['with_links'] += 1
                        
                        if 'website' in json_data:
                            stats['with_website'] += 1
                        
                        # Identifier les profils riches (avec beaucoup de métadonnées)
                        if field_count >= 8 and len(examples['rich_profiles']) < 5:
                            examples['rich_profiles'].append({
                                'name': json_data.get('name', 'N/A'),
                                'id': parts[1],
                                'field_count': field_count,
                                'fields': list(json_data.keys()),
                                'full_object': json_data
                            })
                        
                        # Afficher le progrès
                        if stats['total_authors'] % 1000 == 0:
                            print(f"   Analysé {stats['total_authors']:,} auteurs... (Bio: {stats['with_bio']}, Dates: {stats['with_birth_date']}, Wikipedia: {stats['with_wikipedia']})")
                    
                    except json.JSONDecodeError:
                        continue
    
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")
        return None, None, None
    
    return stats, examples, all_fields

def display_metadata_analysis(stats, examples, all_fields):
    """Affiche les résultats de l'analyse des métadonnées"""
    print(f"\n📊 RÉSULTATS DE L'ANALYSE DES MÉTADONNÉES")
    print("=" * 60)
    
    total = stats['total_authors']
    print(f"👥 Total d'auteurs analysés : {total:,}")
    print(f"\n📈 PRÉSENCE DES MÉTADONNÉES :")
    print(f"   📚 Auteurs avec biographie     : {stats['with_bio']:,} ({stats['with_bio']/total*100:.1f}%)")
    print(f"   🎂 Auteurs avec date naissance : {stats['with_birth_date']:,} ({stats['with_birth_date']/total*100:.1f}%)")
    print(f"   ⚰️ Auteurs avec date décès     : {stats['with_death_date']:,} ({stats['with_death_date']/total*100:.1f}%)")
    print(f"   🌐 Auteurs avec Wikipedia      : {stats['with_wikipedia']:,} ({stats['with_wikipedia']/total*100:.1f}%)")
    print(f"   🔗 Auteurs avec Wikidata       : {stats['with_wikidata']:,} ({stats['with_wikidata']/total*100:.1f}%)")
    print(f"   🔄 Auteurs avec noms alternatifs: {stats['with_alternate_names']:,} ({stats['with_alternate_names']/total*100:.1f}%)")
    print(f"   📝 Auteurs avec nom personnel  : {stats['with_personal_name']:,} ({stats['with_personal_name']/total*100:.1f}%)")
    print(f"   📸 Auteurs avec photos         : {stats['with_photos']:,} ({stats['with_photos']/total*100:.1f}%)")
    print(f"   🔗 Auteurs avec liens externes : {stats['with_links']:,} ({stats['with_links']/total*100:.1f}%)")
    print(f"   🌐 Auteurs avec site web       : {stats['with_website']:,} ({stats['with_website']/total*100:.1f}%)")
    
    print(f"\n📋 CHAMPS LES PLUS FRÉQUENTS :")
    sorted_fields = sorted(all_fields.items(), key=lambda x: x[1], reverse=True)
    for field, count in sorted_fields[:15]:
        percentage = (count / total) * 100
        print(f"   • {field:<25} : {count:>6,} auteurs ({percentage:>5.1f}%)")
    
    # Afficher des exemples
    if examples['bio_examples']:
        print(f"\n📚 EXEMPLES D'AUTEURS AVEC BIOGRAPHIES :")
        for i, ex in enumerate(examples['bio_examples'], 1):
            print(f"   {i}. {ex['name']} ({ex['id']})")
            print(f"      Bio: {ex['bio']}")
    
    if examples['birth_date_examples']:
        print(f"\n🎂 EXEMPLES D'AUTEURS AVEC DATES DE NAISSANCE :")
        for i, ex in enumerate(examples['birth_date_examples'], 1):
            print(f"   {i}. {ex['name']} ({ex['id']})")
            print(f"      Né(e) le: {ex['birth_date']}")
    
    if examples['wikipedia_examples']:
        print(f"\n🌐 EXEMPLES D'AUTEURS AVEC WIKIPEDIA :")
        for i, ex in enumerate(examples['wikipedia_examples'], 1):
            print(f"   {i}. {ex['name']} ({ex['id']})")
            print(f"      Wikipedia: {ex['wikipedia']}")
    
    if examples['wikidata_examples']:
        print(f"\n🔗 EXEMPLES D'AUTEURS AVEC WIKIDATA :")
        for i, ex in enumerate(examples['wikidata_examples'], 1):
            print(f"   {i}. {ex['name']} ({ex['id']})")
            print(f"      Wikidata: {ex['wikidata']}")
    
    if examples['rich_profiles']:
        print(f"\n⭐ EXEMPLES DE PROFILS RICHES (beaucoup de métadonnées) :")
        for i, ex in enumerate(examples['rich_profiles'], 1):
            print(f"   {i}. {ex['name']} ({ex['id']}) - {ex['field_count']} champs")
            print(f"      Champs: {', '.join(ex['fields'][:8])}{'...' if len(ex['fields']) > 8 else ''}")

def display_rich_profile_details(examples):
    """Affiche les détails complets d'un profil riche"""
    if examples['rich_profiles']:
        print(f"\n🔍 DÉTAIL D'UN PROFIL RICHE COMPLET :")
        print("=" * 60)
        
        rich_author = examples['rich_profiles'][0]  # Premier profil riche
        print(f"👤 Auteur: {rich_author['name']}")
        print(f"🆔 ID: {rich_author['id']}")
        print(f"📊 Nombre de champs: {rich_author['field_count']}")
        
        print(f"\n📋 OBJET JSON COMPLET :")
        print("-" * 50)
        try:
            formatted_json = json.dumps(rich_author['full_object'], indent=2, ensure_ascii=False)
            if len(formatted_json) > 2000:
                lines = formatted_json.split('\n')
                print('\n'.join(lines[:60]))
                print(f"\n... [TRONQUÉ - {len(lines)-60} lignes supplémentaires] ...")
                print('\n'.join(lines[-5:]))
            else:
                print(formatted_json)
        except:
            print("❌ Erreur d'affichage du JSON")
        print("-" * 50)

# EXÉCUTION DE L'ANALYSE
fichier_auteurs = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_dump_authors_2025-05-31.txt\ol_dump_authors_2025-05-31.txt"

if os.path.exists(fichier_auteurs):
    print(f"✅ Fichier trouvé : {os.path.basename(fichier_auteurs)}")
    print(f"📁 Analyse en cours sur un échantillon de 50,000 auteurs...")
    
    # Analyser un échantillon plus large
    stats, examples, all_fields = search_author_metadata(fichier_auteurs, sample_size=50000)
    
    if stats:
        display_metadata_analysis(stats, examples, all_fields)
        display_rich_profile_details(examples)
        
        print(f"\n💡 CONCLUSIONS SUR LES MÉTADONNÉES D'AUTEURS :")
        print("=" * 60)
        
        bio_rate = stats['with_bio'] / stats['total_authors'] * 100
        date_rate = stats['with_birth_date'] / stats['total_authors'] * 100
        wiki_rate = stats['with_wikipedia'] / stats['total_authors'] * 100
        wikidata_rate = stats['with_wikidata'] / stats['total_authors'] * 100
        
        if bio_rate > 10:
            print(f"✅ EXCELLENT : {bio_rate:.1f}% des auteurs ont une biographie")
        elif bio_rate > 5:
            print(f"✅ BON : {bio_rate:.1f}% des auteurs ont une biographie")
        else:
            print(f"⚠️ LIMITÉ : Seulement {bio_rate:.1f}% des auteurs ont une biographie")
        
        if date_rate > 20:
            print(f"✅ EXCELLENT : {date_rate:.1f}% des auteurs ont une date de naissance")
        elif date_rate > 10:
            print(f"✅ BON : {date_rate:.1f}% des auteurs ont une date de naissance")
        else:
            print(f"⚠️ LIMITÉ : Seulement {date_rate:.1f}% des auteurs ont une date de naissance")
        
        total_external = wiki_rate + wikidata_rate
        if total_external > 30:
            print(f"✅ EXCELLENT : {total_external:.1f}% des auteurs ont des liens externes (Wikipedia/Wikidata)")
        elif total_external > 15:
            print(f"✅ BON : {total_external:.1f}% des auteurs ont des liens externes")
        else:
            print(f"⚠️ LIMITÉ : Seulement {total_external:.1f}% des auteurs ont des liens externes")
        
        print(f"\n🎯 RECOMMANDATIONS :")
        rich_rate = len(examples['rich_profiles']) / stats['total_authors'] * 100
        if rich_rate > 0.1:
            print(f"💡 Il existe des profils riches ({rich_rate:.2f}%) - concentrez-vous sur ceux-ci pour l'analyse")
        
        if bio_rate > 5:
            print(f"📚 Les biographies sont présentes - utilisez-les pour l'analyse de contenu")
        
        if date_rate > 10:
            print(f"📅 Les dates sont disponibles - possibilité d'analyse temporelle")
        
        if wiki_rate > 5 or wikidata_rate > 5:
            print(f"🔗 Liens externes disponibles - possibilité d'enrichissement des données")
        
    else:
        print("❌ Échec de l'analyse des métadonnées")
else:
    print("❌ Fichier d'auteurs non trouvé")
    print(f"Chemin recherché : {fichier_auteurs}")


🔍 RECHERCHE APPROFONDIE DES MÉTADONNÉES D'AUTEURS
✅ Fichier trouvé : ol_dump_authors_2025-05-31.txt
📁 Analyse en cours sur un échantillon de 50,000 auteurs...
🔎 Analyse de 50,000 auteurs pour trouver les métadonnées...
   Analysé 1,000 auteurs... (Bio: 1, Dates: 31, Wikipedia: 0)
   Analysé 2,000 auteurs... (Bio: 1, Dates: 80, Wikipedia: 0)
   Analysé 3,000 auteurs... (Bio: 2, Dates: 126, Wikipedia: 0)
   Analysé 4,000 auteurs... (Bio: 5, Dates: 166, Wikipedia: 0)
   Analysé 5,000 auteurs... (Bio: 5, Dates: 190, Wikipedia: 0)
   Analysé 6,000 auteurs... (Bio: 6, Dates: 223, Wikipedia: 0)
   Analysé 7,000 auteurs... (Bio: 7, Dates: 331, Wikipedia: 0)
   Analysé 8,000 auteurs... (Bio: 7, Dates: 557, Wikipedia: 0)
   Analysé 9,000 auteurs... (Bio: 7, Dates: 784, Wikipedia: 0)
   Analysé 10,000 auteurs... (Bio: 11, Dates: 878, Wikipedia: 0)
   Analysé 11,000 auteurs... (Bio: 11, Dates: 914, Wikipedia: 0)
   Analysé 12,000 auteurs... (Bio: 11, Dates: 943, Wikipedia: 0)
   Analysé 13,000 aut

In [20]:
# EXTRACTION DES AUTEURS AVEC DATE DE NAISSANCE
print("🎂 EXTRACTION DES AUTEURS AVEC DATE DE NAISSANCE")
print("=" * 60)

import pandas as pd
import json
import os
from datetime import datetime
import re

def extract_authors_with_birth_date(filename, max_authors=None):
    """Extrait tous les auteurs ayant une date de naissance"""
    print(f"🔍 Recherche des auteurs avec date de naissance...")
    
    authors_data = []
    total_processed = 0
    authors_found = 0
    
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if max_authors and authors_found >= max_authors:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5 and parts[0] == '/type/author':
                    total_processed += 1
                    
                    try:
                        json_data = json.loads(parts[4])
                        
                        # Vérifier si l'auteur a une date de naissance
                        if 'birth_date' in json_data:
                            birth_date = json_data['birth_date']
                            
                            # Préparer les données de l'auteur
                            author_info = {
                                'author_id': parts[1],
                                'name': json_data.get('name', ''),
                                'birth_date': birth_date,
                                'death_date': json_data.get('death_date', ''),
                                'personal_name': json_data.get('personal_name', ''),
                                'bio': '',
                                'wikipedia': json_data.get('wikipedia', ''),
                                'wikidata': '',
                                'alternate_names': '',
                                'photos': '',
                                'website': json_data.get('website', ''),
                                'created': parts[3],  # timestamp
                                'revision': parts[2],
                                'birth_year': None,
                                'death_year': None,
                                'age_at_death': None,
                                'is_alive': None
                            }
                            
                            # Extraire la biographie
                            if 'bio' in json_data:
                                bio = json_data['bio']
                                if isinstance(bio, dict) and 'value' in bio:
                                    author_info['bio'] = bio['value']
                                elif isinstance(bio, str):
                                    author_info['bio'] = bio
                            
                            # Extraire Wikidata depuis remote_ids
                            if 'remote_ids' in json_data:
                                remote_ids = json_data['remote_ids']
                                if isinstance(remote_ids, dict) and 'wikidata' in remote_ids:
                                    author_info['wikidata'] = remote_ids['wikidata']
                            
                            # Extraire les noms alternatifs
                            if 'alternate_names' in json_data and json_data['alternate_names']:
                                if isinstance(json_data['alternate_names'], list):
                                    author_info['alternate_names'] = ' | '.join(json_data['alternate_names'])
                            
                            # Extraire les photos
                            if 'photos' in json_data and json_data['photos']:
                                if isinstance(json_data['photos'], list):
                                    author_info['photos'] = ' | '.join([str(photo) for photo in json_data['photos']])
                            
                            # Traiter les dates pour extraire les années
                            try:
                                # Extraire l'année de naissance
                                birth_match = re.search(r'\b(\d{4})\b', str(birth_date))
                                if birth_match:
                                    birth_year = int(birth_match.group(1))
                                    if 1000 <= birth_year <= datetime.now().year:
                                        author_info['birth_year'] = birth_year
                                
                                # Extraire l'année de décès si présente
                                if author_info['death_date']:
                                    death_match = re.search(r'\b(\d{4})\b', str(author_info['death_date']))
                                    if death_match:
                                        death_year = int(death_match.group(1))
                                        if 1000 <= death_year <= datetime.now().year:
                                            author_info['death_year'] = death_year
                                            
                                            # Calculer l'âge au décès
                                            if author_info['birth_year']:
                                                author_info['age_at_death'] = death_year - author_info['birth_year']
                                                author_info['is_alive'] = False
                                        else:
                                            author_info['is_alive'] = True
                                else:
                                    # Pas de date de décès = probablement vivant
                                    if author_info['birth_year']:
                                        current_year = datetime.now().year
                                        age_now = current_year - author_info['birth_year']
                                        # Si la personne aurait plus de 120 ans, elle est probablement décédée
                                        author_info['is_alive'] = age_now <= 120
                            
                            except Exception:
                                pass  # Ignorer les erreurs de parsing de dates
                            
                            authors_data.append(author_info)
                            authors_found += 1
                    
                    except json.JSONDecodeError:
                        continue
                
                # Afficher le progrès
                if total_processed % 10000 == 0:
                    print(f"   Traité {total_processed:,} auteurs, trouvé {authors_found:,} avec date de naissance...")
    
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        return None
    
    print(f"✅ Extraction terminée : {authors_found:,} auteurs avec date de naissance trouvés sur {total_processed:,} auteurs traités")
    return authors_data


🎂 EXTRACTION DES AUTEURS AVEC DATE DE NAISSANCE


In [21]:
def create_authors_dataframe(authors_data):
    """Crée un DataFrame pandas à partir des données d'auteurs"""
    print(f"📊 Création du DataFrame avec {len(authors_data):,} auteurs...")
    
    df = pd.DataFrame(authors_data)
    
    # Statistiques sur le DataFrame
    print(f"\n📈 STATISTIQUES DU DATAFRAME :")
    print(f"   📏 Dimensions : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
    print(f"   🎂 Années de naissance : {df['birth_year'].min()} - {df['birth_year'].max()}")
    
    # Répartition par siècle
    if 'birth_year' in df.columns and df['birth_year'].notna().sum() > 0:
        birth_years = df['birth_year'].dropna()
        print(f"\n🏛️ RÉPARTITION PAR SIÈCLE :")
        for century in range(15, 22):  # 15ème au 21ème siècle
            start_year = (century - 1) * 100 + 1
            end_year = century * 100
            count = birth_years[(birth_years >= start_year) & (birth_years <= end_year)].count()
            if count > 0:
                print(f"   • {century}ème siècle ({start_year}-{end_year}) : {count:,} auteurs")
    
    # Taux de remplissage des champs
    print(f"\n📋 TAUX DE REMPLISSAGE DES CHAMPS :")
    for col in df.columns:
        if col in ['bio', 'wikipedia', 'wikidata', 'alternate_names', 'death_date', 'photos', 'website']:
            non_empty = df[col].notna() & (df[col] != '')
            percentage = (non_empty.sum() / len(df)) * 100
            print(f"   • {col:<20} : {non_empty.sum():>6,} ({percentage:>5.1f}%)")
    
    # Auteurs vivants vs décédés
    if 'is_alive' in df.columns:
        alive_count = df['is_alive'].sum()
        dead_count = (df['is_alive'] == False).sum()
        unknown_count = df['is_alive'].isna().sum()
        print(f"\n👥 STATUT VITAL :")
        print(f"   🟢 Probablement vivants : {alive_count:,}")
        print(f"   🔴 Probablement décédés : {dead_count:,}")
        print(f"   ❓ Statut inconnu       : {unknown_count:,}")
    
    return df

def display_dataframe_sample(df, n=10):
    """Affiche un échantillon du DataFrame"""
    print(f"\n📋 APERÇU DU DATAFRAME (premiers {n} auteurs) :")
    print("=" * 80)
    
    # Sélectionner les colonnes les plus importantes pour l'affichage
    display_cols = ['name', 'birth_year', 'death_year', 'age_at_death', 'is_alive']
    available_cols = [col for col in display_cols if col in df.columns]
    
    print(df[available_cols].head(n).to_string(index=False))
    
    print(f"\n🔍 QUELQUES EXEMPLES INTÉRESSANTS :")
    
    # Auteurs les plus anciens
    if 'birth_year' in df.columns:
        oldest = df.nsmallest(3, 'birth_year')[['name', 'birth_year', 'death_year']]
        print(f"\n👴 AUTEURS LES PLUS ANCIENS :")
        for _, author in oldest.iterrows():
            death_info = f" (mort en {author['death_year']:.0f})" if pd.notna(author['death_year']) else " (date de décès inconnue)"
            print(f"   • {author['name']} - né en {author['birth_year']:.0f}{death_info}")
    
    # Auteurs les plus récents
    if 'birth_year' in df.columns:
        youngest = df.nlargest(3, 'birth_year')[['name', 'birth_year', 'is_alive']]
        print(f"\n👶 AUTEURS LES PLUS RÉCENTS :")
        for _, author in youngest.iterrows():
            status = " (probablement vivant)" if author['is_alive'] else " (statut inconnu)"
            print(f"   • {author['name']} - né en {author['birth_year']:.0f}{status}")
    
    # Auteurs avec biographie
    if 'bio' in df.columns:
        with_bio = df[df['bio'].str.len() > 50].head(3)
        if not with_bio.empty:
            print(f"\n📚 AUTEURS AVEC BIOGRAPHIE :")
            for _, author in with_bio.iterrows():
                bio_preview = author['bio'][:100] + "..." if len(author['bio']) > 100 else author['bio']
                print(f"   • {author['name']} : {bio_preview}")


In [23]:
# EXÉCUTION DE L'EXTRACTION
fichier_auteurs = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_dump_authors_2025-05-31.txt\ol_dump_authors_2025-05-31.txt"

if os.path.exists(fichier_auteurs):
    print(f"✅ Fichier trouvé : {os.path.basename(fichier_auteurs)}")
    
    # Demander à l'utilisateur s'il veut limiter le nombre d'auteurs
    print(f"\n⚠️ ATTENTION : Le fichier complet peut contenir des millions d'auteurs.")
    print(f"Pour un test rapide, vous pouvez limiter à un échantillon.")
    print(f"\nOptions :")
    print(f"  1. Échantillon rapide (10,000 premiers auteurs avec date de naissance)")
    print(f"  2. Échantillon moyen (50,000 premiers auteurs avec date de naissance)")
    print(f"  3. Extraction complète (TOUS les auteurs - peut prendre du temps)")
    
    # Pour cet exemple, on prend un échantillon moyen
    choice = 3  # Vous pouvez changer cette valeur
    
    if choice == 1:
        max_authors = 10000
        print(f"\n🚀 Extraction d'un échantillon rapide (max {max_authors:,} auteurs)...")
    elif choice == 2:
        max_authors = 50000
        print(f"\n🚀 Extraction d'un échantillon moyen (max {max_authors:,} auteurs)...")
    else:
        max_authors = None
        print(f"\n🚀 Extraction complète (TOUS les auteurs avec date de naissance)...")
    
    # Extraire les données
    authors_data = extract_authors_with_birth_date(fichier_auteurs, max_authors=max_authors)
    
    if authors_data:
        # Créer le DataFrame
        df_authors_birth = create_authors_dataframe(authors_data)
        
        # Afficher un aperçu
        display_dataframe_sample(df_authors_birth, n=10)
        
        print(f"\n💾 SAUVEGARDE DU DATAFRAME :")
        print(f"Le DataFrame est maintenant disponible dans la variable 'df_authors_birth'")
        print(f"\n📁 Pour sauvegarder en CSV :")
        print(f"   df_authors_birth.to_csv('auteurs_avec_date_naissance.csv', index=False)")
        
        print(f"\n🔍 UTILISATION DU DATAFRAME :")
        print(f"   • df_authors_birth.info() - Informations sur le DataFrame")
        print(f"   • df_authors_birth.describe() - Statistiques descriptives")
        print(f"   • df_authors_birth[df_authors_birth['is_alive'] == True] - Auteurs vivants")
        print(f"   • df_authors_birth[df_authors_birth['bio'] != ''] - Auteurs avec biographie")
        print(f"   • df_authors_birth.groupby('birth_year').size() - Répartition par année")
    
    else:
        print("❌ Aucune donnée d'auteur extraite")
else:
    print("❌ Fichier d'auteurs non trouvé")
    print(f"Chemin recherché : {fichier_auteurs}")


✅ Fichier trouvé : ol_dump_authors_2025-05-31.txt

⚠️ ATTENTION : Le fichier complet peut contenir des millions d'auteurs.
Pour un test rapide, vous pouvez limiter à un échantillon.

Options :
  1. Échantillon rapide (10,000 premiers auteurs avec date de naissance)
  2. Échantillon moyen (50,000 premiers auteurs avec date de naissance)
  3. Extraction complète (TOUS les auteurs - peut prendre du temps)

🚀 Extraction complète (TOUS les auteurs avec date de naissance)...
🔍 Recherche des auteurs avec date de naissance...
   Traité 10,000 auteurs, trouvé 878 avec date de naissance...
   Traité 20,000 auteurs, trouvé 1,696 avec date de naissance...
   Traité 30,000 auteurs, trouvé 3,525 avec date de naissance...
   Traité 40,000 auteurs, trouvé 4,885 avec date de naissance...
   Traité 50,000 auteurs, trouvé 6,343 avec date de naissance...
   Traité 60,000 auteurs, trouvé 7,226 avec date de naissance...
   Traité 70,000 auteurs, trouvé 8,030 avec date de naissance...
   Traité 80,000 auteur

In [31]:
df_authors_birth.to_csv('auteurs_avec_date_naissance.csv', index=False)

In [27]:
# EXTRACTION DES INFORMATIONS SUR LES LIVRES
print("📚 EXTRACTION DES INFORMATIONS SUR LES LIVRES")
print("=" * 60)

import pandas as pd
import json
import os
import gzip
from datetime import datetime
import re

def analyze_book_files():
    """Analyse les fichiers disponibles pour les livres"""
    base_path = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary"
    
    print("🔍 RECHERCHE DES FICHIERS DE LIVRES DISPONIBLES :")
    print("-" * 50)
    
    book_files = []
    
    # Rechercher les fichiers d'éditions et de works
    for filename in os.listdir(base_path):
        if 'edition' in filename.lower() or 'work' in filename.lower():
            file_path = os.path.join(base_path, filename)
            file_size = os.path.getsize(file_path)
            file_size_mb = file_size / (1024 * 1024)
            
            book_files.append({
                'filename': filename,
                'path': file_path,
                'size_mb': file_size_mb,
                'type': 'editions' if 'edition' in filename.lower() else 'works'
            })
            
            print(f"📁 {filename}")
            print(f"   Taille : {file_size_mb:.1f} MB")
            print(f"   Type : {'Éditions' if 'edition' in filename.lower() else 'Œuvres'}")
            print()
    
    return book_files

def extract_books_sample(filename, max_books=1000, book_type='editions'):
    """Extrait un échantillon de livres pour analyser la structure"""
    print(f"📖 ANALYSE D'UN ÉCHANTILLON DE {book_type.upper()}")
    print(f"   Fichier : {os.path.basename(filename)}")
    print(f"   Échantillon : {max_books:,} entrées")
    print("-" * 50)
    
    books_data = []
    total_processed = 0
    books_found = 0
    
    try:
        # Déterminer si le fichier est compressé
        open_func = gzip.open if filename.endswith('.gz') else open
        mode = 'rt' if filename.endswith('.gz') else 'r'
        
        with open_func(filename, mode, encoding='utf-8') as f:
            for i, line in enumerate(f):
                if books_found >= max_books:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5:
                    entry_type = parts[0]
                    
                    # Filtrer selon le type de livre recherché
                    if (book_type == 'editions' and entry_type == '/type/edition') or \
                       (book_type == 'works' and entry_type == '/type/work'):
                        
                        total_processed += 1
                        
                        try:
                            json_data = json.loads(parts[4])
                            
                            # Extraire les informations de base
                            book_info = {
                                'entry_type': entry_type,
                                'book_id': parts[1],
                                'revision': parts[2],
                                'created': parts[3],
                                'title': json_data.get('title', ''),
                                'subtitle': json_data.get('subtitle', ''),
                            }
                            
                            # Informations spécifiques aux éditions
                            if book_type == 'editions':
                                book_info.update({
                                    'isbn_10': '',
                                    'isbn_13': '',
                                    'publishers': '',
                                    'publish_date': json_data.get('publish_date', ''),
                                    'publish_year': None,
                                    'pages': json_data.get('number_of_pages', ''),
                                    'languages': '',
                                    'authors': '',
                                    'works': '',
                                    'edition_name': json_data.get('edition_name', ''),
                                    'physical_format': json_data.get('physical_format', ''),
                                    'weight': json_data.get('weight', ''),
                                    'dimensions': json_data.get('dimensions', ''),
                                })
                                
                                # Extraire ISBN
                                if 'isbn_10' in json_data:
                                    book_info['isbn_10'] = ' | '.join(json_data['isbn_10']) if isinstance(json_data['isbn_10'], list) else str(json_data['isbn_10'])
                                if 'isbn_13' in json_data:
                                    book_info['isbn_13'] = ' | '.join(json_data['isbn_13']) if isinstance(json_data['isbn_13'], list) else str(json_data['isbn_13'])
                                
                                # Extraire éditeurs
                                if 'publishers' in json_data:
                                    book_info['publishers'] = ' | '.join(json_data['publishers']) if isinstance(json_data['publishers'], list) else str(json_data['publishers'])
                                
                                # Extraire langues
                                if 'languages' in json_data:
                                    if isinstance(json_data['languages'], list):
                                        languages = []
                                        for lang in json_data['languages']:
                                            if isinstance(lang, dict) and 'key' in lang:
                                                languages.append(lang['key'].replace('/languages/', ''))
                                            else:
                                                languages.append(str(lang))
                                        book_info['languages'] = ' | '.join(languages)
                                
                                # Extraire auteurs
                                if 'authors' in json_data:
                                    if isinstance(json_data['authors'], list):
                                        authors = []
                                        for author in json_data['authors']:
                                            if isinstance(author, dict) and 'key' in author:
                                                authors.append(author['key'])
                                            else:
                                                authors.append(str(author))
                                        book_info['authors'] = ' | '.join(authors)
                                
                                # Extraire works
                                if 'works' in json_data:
                                    if isinstance(json_data['works'], list):
                                        works = []
                                        for work in json_data['works']:
                                            if isinstance(work, dict) and 'key' in work:
                                                works.append(work['key'])
                                            else:
                                                works.append(str(work))
                                        book_info['works'] = ' | '.join(works)
                                
                                # Extraire l'année de publication
                                if book_info['publish_date']:
                                    year_match = re.search(r'\b(\d{4})\b', str(book_info['publish_date']))
                                    if year_match:
                                        year = int(year_match.group(1))
                                        if 1000 <= year <= datetime.now().year:
                                            book_info['publish_year'] = year
                            
                            # Informations spécifiques aux works (œuvres)
                            elif book_type == 'works':
                                book_info.update({
                                    'authors': '',
                                    'subjects': '',
                                    'subject_places': '',
                                    'subject_times': '',
                                    'subject_people': '',
                                    'description': '',
                                    'first_publish_date': json_data.get('first_publish_date', ''),
                                    'first_publish_year': None,
                                    'covers': '',
                                    'excerpts': '',
                                    'links': '',
                                })
                                
                                # Extraire auteurs
                                if 'authors' in json_data:
                                    if isinstance(json_data['authors'], list):
                                        authors = []
                                        for author in json_data['authors']:
                                            if isinstance(author, dict):
                                                if 'author' in author and isinstance(author['author'], dict) and 'key' in author['author']:
                                                    authors.append(author['author']['key'])
                                                elif 'key' in author:
                                                    authors.append(author['key'])
                                            else:
                                                authors.append(str(author))
                                        book_info['authors'] = ' | '.join(authors)
                                
                                # Extraire sujets
                                if 'subjects' in json_data:
                                    book_info['subjects'] = ' | '.join(json_data['subjects']) if isinstance(json_data['subjects'], list) else str(json_data['subjects'])
                                
                                # Extraire description
                                if 'description' in json_data:
                                    desc = json_data['description']
                                    if isinstance(desc, dict) and 'value' in desc:
                                        book_info['description'] = desc['value']
                                    elif isinstance(desc, str):
                                        book_info['description'] = desc
                                
                                # Extraire l'année de première publication
                                if book_info['first_publish_date']:
                                    year_match = re.search(r'\b(\d{4})\b', str(book_info['first_publish_date']))
                                    if year_match:
                                        year = int(year_match.group(1))
                                        if 1000 <= year <= datetime.now().year:
                                            book_info['first_publish_year'] = year
                            
                            books_data.append(book_info)
                            books_found += 1
                        
                        except json.JSONDecodeError:
                            continue
                
                # Afficher le progrès
                if total_processed % 1000 == 0 and total_processed > 0:
                    print(f"   Traité {total_processed:,} entrées, trouvé {books_found:,} {book_type}...")
    
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        return None
    
    print(f"✅ Échantillon extrait : {books_found:,} {book_type} sur {total_processed:,} entrées traitées")
    return books_data

def analyze_books_structure(books_data, book_type='editions'):
    """Analyse la structure des données de livres"""
    if not books_data:
        return
    
    print(f"\n📊 ANALYSE DE LA STRUCTURE DES {book_type.upper()}")
    print("=" * 60)
    
    df = pd.DataFrame(books_data)
    
    print(f"📏 Dimensions : {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
    print(f"\n📋 COLONNES DISPONIBLES :")
    for i, col in enumerate(df.columns, 1):
        non_empty = df[col].notna() & (df[col] != '')
        percentage = (non_empty.sum() / len(df)) * 100
        print(f"   {i:2d}. {col:<20} : {non_empty.sum():>5,} ({percentage:>5.1f}%)")
    
    # Analyse spécifique selon le type
    if book_type == 'editions':
        print(f"\n📚 ANALYSE SPÉCIFIQUE DES ÉDITIONS :")
        
        # Années de publication
        if 'publish_year' in df.columns:
            years = df['publish_year'].dropna()
            if len(years) > 0:
                print(f"   📅 Années de publication : {years.min():.0f} - {years.max():.0f}")
                
                # Répartition par décennie récente
                recent_years = years[years >= 1900]
                if len(recent_years) > 0:
                    print(f"   📈 Publications depuis 1900 : {len(recent_years):,}")
                    for decade in range(1900, 2030, 10):
                        count = recent_years[(recent_years >= decade) & (recent_years < decade + 10)].count()
                        if count > 0:
                            print(f"      • {decade}s : {count:,} éditions")
        
        # Langues les plus fréquentes
        if 'languages' in df.columns:
            all_languages = []
            for langs in df['languages'].dropna():
                if langs:
                    all_languages.extend(langs.split(' | '))
            
            if all_languages:
                lang_counts = pd.Series(all_languages).value_counts().head(10)
                print(f"\n   🌍 LANGUES LES PLUS FRÉQUENTES :")
                for lang, count in lang_counts.items():
                    print(f"      • {lang:<15} : {count:,} éditions")
        
        # Éditeurs les plus fréquents
        if 'publishers' in df.columns:
            all_publishers = []
            for pubs in df['publishers'].dropna():
                if pubs:
                    all_publishers.extend(pubs.split(' | '))
            
            if all_publishers:
                pub_counts = pd.Series(all_publishers).value_counts().head(10)
                print(f"\n   🏢 ÉDITEURS LES PLUS FRÉQUENTS :")
                for pub, count in pub_counts.items():
                    print(f"      • {pub:<30} : {count:,} éditions")
    
    elif book_type == 'works':
        print(f"\n📖 ANALYSE SPÉCIFIQUE DES ŒUVRES :")
        
        # Années de première publication
        if 'first_publish_year' in df.columns:
            years = df['first_publish_year'].dropna()
            if len(years) > 0:
                print(f"   📅 Premières publications : {years.min():.0f} - {years.max():.0f}")
        
        # Sujets les plus fréquents
        if 'subjects' in df.columns:
            all_subjects = []
            for subjects in df['subjects'].dropna():
                if subjects:
                    all_subjects.extend(subjects.split(' | '))
            
            if all_subjects:
                subject_counts = pd.Series(all_subjects).value_counts().head(15)
                print(f"\n   🏷️ SUJETS LES PLUS FRÉQUENTS :")
                for subject, count in subject_counts.items():
                    print(f"      • {subject:<35} : {count:,} œuvres")
    
    # Exemples de livres
    print(f"\n📋 EXEMPLES DE {book_type.upper()} :")
    print("-" * 60)
    
    display_cols = ['title', 'publish_year' if book_type == 'editions' else 'first_publish_year']
    if book_type == 'editions':
        display_cols.extend(['publishers', 'languages'])
    else:
        display_cols.extend(['subjects'])
    
    available_cols = [col for col in display_cols if col in df.columns]
    
    sample_df = df[available_cols].head(5)
    for _, book in sample_df.iterrows():
        print(f"📚 {book['title']}")
        for col in available_cols[1:]:  # Skip title
            if pd.notna(book[col]) and book[col] != '':
                value = str(book[col])[:100] + "..." if len(str(book[col])) > 100 else str(book[col])
                print(f"   {col}: {value}")
        print()
    
    return df

# LANCEMENT DE L'ANALYSE
print("🚀 DÉMARRAGE DE L'ANALYSE DES LIVRES")
print("=" * 60)

# 1. Analyser les fichiers disponibles
book_files = analyze_book_files()

if book_files:
    print(f"✅ {len(book_files)} fichier(s) de livres trouvé(s)")
    
    # 2. Choisir le fichier à analyser (prendre le premier fichier d'éditions)
    editions_files = [f for f in book_files if f['type'] == 'editions']
    works_files = [f for f in book_files if f['type'] == 'works']
    
    if editions_files:
        print(f"\n📚 ANALYSE DES ÉDITIONS")
        selected_file = editions_files[0]
        print(f"   Fichier sélectionné : {selected_file['filename']}")
        
        # Extraire un échantillon d'éditions
        books_sample = extract_books_sample(selected_file['path'], max_books=2000, book_type='editions')
        
        if books_sample:
            # Analyser la structure
            df_books = analyze_books_structure(books_sample, book_type='editions')
            print(f"\n💾 DataFrame 'df_books' créé avec {len(books_sample):,} éditions")
        
    if works_files:
        print(f"\n📖 ANALYSE DES ŒUVRES")
        selected_file = works_files[0]
        print(f"   Fichier sélectionné : {selected_file['filename']}")
        
        # Extraire un échantillon d'œuvres
        works_sample = extract_books_sample(selected_file['path'], max_books=2000, book_type='works')
        
        if works_sample:
            # Analyser la structure
            df_works = analyze_books_structure(works_sample, book_type='works')
            print(f"\n💾 DataFrame 'df_works' créé avec {len(works_sample):,} œuvres")

else:
    print("❌ Aucun fichier de livres trouvé")


📚 EXTRACTION DES INFORMATIONS SUR LES LIVRES
🚀 DÉMARRAGE DE L'ANALYSE DES LIVRES
🔍 RECHERCHE DES FICHIERS DE LIVRES DISPONIBLES :
--------------------------------------------------
📁 autrices-auteurs-collections-maisons-edition-2018-05-01.csv
   Taille : 0.2 MB
   Type : Éditions

📁 ol_dump_editions_2025-05-31.txt.gz
   Taille : 10969.8 MB
   Type : Éditions

✅ 2 fichier(s) de livres trouvé(s)

📚 ANALYSE DES ÉDITIONS
   Fichier sélectionné : autrices-auteurs-collections-maisons-edition-2018-05-01.csv
📖 ANALYSE D'UN ÉCHANTILLON DE EDITIONS
   Fichier : autrices-auteurs-collections-maisons-edition-2018-05-01.csv
   Échantillon : 2,000 entrées
--------------------------------------------------
✅ Échantillon extrait : 0 editions sur 0 entrées traitées


In [28]:
# EXTRACTION SPÉCIALISÉE DES LIVRES
print("🎯 EXTRACTION SPÉCIALISÉE DES LIVRES")
print("=" * 60)

def extract_books_with_criteria(filename, max_books=10000, criteria=None):
    """Extrait les livres selon des critères spécifiques"""
    if criteria is None:
        criteria = {
            'has_isbn': True,
            'has_authors': True,
            'min_year': 1900,
            'max_year': 2024,
            'languages': ['eng', 'fre', 'spa', 'ger', 'ita'],  # Langues principales
        }
    
    print(f"🔍 EXTRACTION AVEC CRITÈRES :")
    print(f"   • ISBN requis : {criteria.get('has_isbn', False)}")
    print(f"   • Auteurs requis : {criteria.get('has_authors', False)}")
    print(f"   • Années : {criteria.get('min_year', 'Toutes')} - {criteria.get('max_year', 'Toutes')}")
    print(f"   • Langues : {', '.join(criteria.get('languages', ['Toutes']))}")
    print(f"   • Maximum : {max_books:,} livres")
    print("-" * 50)
    
    books_data = []
    total_processed = 0
    books_found = 0
    
    try:
        # Déterminer si le fichier est compressé
        open_func = gzip.open if filename.endswith('.gz') else open
        mode = 'rt' if filename.endswith('.gz') else 'r'
        
        with open_func(filename, mode, encoding='utf-8') as f:
            for i, line in enumerate(f):
                if books_found >= max_books:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5 and parts[0] == '/type/edition':
                    total_processed += 1
                    
                    try:
                        json_data = json.loads(parts[4])
                        
                        # Vérifier les critères
                        meets_criteria = True
                        
                        # Critère ISBN
                        if criteria.get('has_isbn', False):
                            has_isbn = 'isbn_10' in json_data or 'isbn_13' in json_data
                            if not has_isbn:
                                meets_criteria = False
                                continue
                        
                        # Critère auteurs
                        if criteria.get('has_authors', False):
                            has_authors = 'authors' in json_data and json_data['authors']
                            if not has_authors:
                                meets_criteria = False
                                continue
                        
                        # Critère année
                        if criteria.get('min_year') or criteria.get('max_year'):
                            publish_year = None
                            if 'publish_date' in json_data:
                                year_match = re.search(r'\b(\d{4})\b', str(json_data['publish_date']))
                                if year_match:
                                    publish_year = int(year_match.group(1))
                            
                            if publish_year:
                                if criteria.get('min_year') and publish_year < criteria['min_year']:
                                    meets_criteria = False
                                    continue
                                if criteria.get('max_year') and publish_year > criteria['max_year']:
                                    meets_criteria = False
                                    continue
                        
                        # Critère langue
                        if criteria.get('languages'):
                            book_languages = []
                            if 'languages' in json_data:
                                for lang in json_data['languages']:
                                    if isinstance(lang, dict) and 'key' in lang:
                                        lang_code = lang['key'].replace('/languages/', '')
                                        book_languages.append(lang_code)
                            
                            if book_languages:
                                has_target_language = any(lang in criteria['languages'] for lang in book_languages)
                                if not has_target_language:
                                    meets_criteria = False
                                    continue
                        
                        if meets_criteria:
                            # Extraire les informations complètes
                            book_info = {
                                'book_id': parts[1],
                                'title': json_data.get('title', ''),
                                'subtitle': json_data.get('subtitle', ''),
                                'isbn_10': '',
                                'isbn_13': '',
                                'publishers': '',
                                'publish_date': json_data.get('publish_date', ''),
                                'publish_year': publish_year,
                                'pages': json_data.get('number_of_pages', ''),
                                'languages': '',
                                'authors': '',
                                'works': '',
                                'physical_format': json_data.get('physical_format', ''),
                                'edition_name': json_data.get('edition_name', ''),
                                'description': '',
                                'subjects': '',
                                'weight': json_data.get('weight', ''),
                                'dimensions': json_data.get('dimensions', ''),
                                'covers': '',
                                'created': parts[3],
                                'revision': parts[2],
                            }
                            
                            # Extraire ISBN
                            if 'isbn_10' in json_data:
                                book_info['isbn_10'] = ' | '.join(json_data['isbn_10']) if isinstance(json_data['isbn_10'], list) else str(json_data['isbn_10'])
                            if 'isbn_13' in json_data:
                                book_info['isbn_13'] = ' | '.join(json_data['isbn_13']) if isinstance(json_data['isbn_13'], list) else str(json_data['isbn_13'])
                            
                            # Extraire éditeurs
                            if 'publishers' in json_data:
                                book_info['publishers'] = ' | '.join(json_data['publishers']) if isinstance(json_data['publishers'], list) else str(json_data['publishers'])
                            
                            # Extraire langues
                            if 'languages' in json_data:
                                languages = []
                                for lang in json_data['languages']:
                                    if isinstance(lang, dict) and 'key' in lang:
                                        languages.append(lang['key'].replace('/languages/', ''))
                                    else:
                                        languages.append(str(lang))
                                book_info['languages'] = ' | '.join(languages)
                            
                            # Extraire auteurs
                            if 'authors' in json_data:
                                authors = []
                                for author in json_data['authors']:
                                    if isinstance(author, dict) and 'key' in author:
                                        authors.append(author['key'])
                                    else:
                                        authors.append(str(author))
                                book_info['authors'] = ' | '.join(authors)
                            
                            # Extraire works
                            if 'works' in json_data:
                                works = []
                                for work in json_data['works']:
                                    if isinstance(work, dict) and 'key' in work:
                                        works.append(work['key'])
                                    else:
                                        works.append(str(work))
                                book_info['works'] = ' | '.join(works)
                            
                            # Extraire description
                            if 'description' in json_data:
                                desc = json_data['description']
                                if isinstance(desc, dict) and 'value' in desc:
                                    book_info['description'] = desc['value']
                                elif isinstance(desc, str):
                                    book_info['description'] = desc
                            
                            # Extraire sujets
                            if 'subjects' in json_data:
                                book_info['subjects'] = ' | '.join(json_data['subjects']) if isinstance(json_data['subjects'], list) else str(json_data['subjects'])
                            
                            # Extraire covers
                            if 'covers' in json_data:
                                book_info['covers'] = ' | '.join([str(cover) for cover in json_data['covers']]) if isinstance(json_data['covers'], list) else str(json_data['covers'])
                            
                            books_data.append(book_info)
                            books_found += 1
                    
                    except json.JSONDecodeError:
                        continue
                
                # Afficher le progrès
                if total_processed % 5000 == 0 and total_processed > 0:
                    print(f"   Traité {total_processed:,} éditions, trouvé {books_found:,} livres correspondants...")
    
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        return None
    
    print(f"✅ Extraction terminée : {books_found:,} livres trouvés sur {total_processed:,} éditions traitées")
    return books_data

def analyze_extracted_books(books_data):
    """Analyse les livres extraits"""
    if not books_data:
        return None
    
    print(f"\n📊 ANALYSE DES LIVRES EXTRAITS")
    print("=" * 60)
    
    df = pd.DataFrame(books_data)
    
    print(f"📏 Dimensions : {df.shape[0]:,} livres × {df.shape[1]} colonnes")
    
    # Analyse des années de publication
    if 'publish_year' in df.columns:
        years = df['publish_year'].dropna()
        if len(years) > 0:
            print(f"\n📅 RÉPARTITION PAR DÉCENNIE :")
            for decade in range(1900, 2030, 10):
                count = years[(years >= decade) & (years < decade + 10)].count()
                if count > 0:
                    percentage = (count / len(years)) * 100
                    print(f"   • {decade}s : {count:,} livres ({percentage:.1f}%)")
    
    # Analyse des langues
    if 'languages' in df.columns:
        all_languages = []
        for langs in df['languages'].dropna():
            if langs:
                all_languages.extend(langs.split(' | '))
        
        if all_languages:
            lang_counts = pd.Series(all_languages).value_counts().head(10)
            print(f"\n🌍 RÉPARTITION PAR LANGUE :")
            for lang, count in lang_counts.items():
                percentage = (count / len(all_languages)) * 100
                print(f"   • {lang:<10} : {count:,} livres ({percentage:.1f}%)")
    
    # Analyse des éditeurs
    if 'publishers' in df.columns:
        all_publishers = []
        for pubs in df['publishers'].dropna():
            if pubs:
                all_publishers.extend(pubs.split(' | '))
        
        if all_publishers:
            pub_counts = pd.Series(all_publishers).value_counts().head(10)
            print(f"\n🏢 TOP 10 ÉDITEURS :")
            for pub, count in pub_counts.items():
                print(f"   • {pub:<35} : {count:,} livres")
    
    # Analyse des formats physiques
    if 'physical_format' in df.columns:
        format_counts = df['physical_format'].value_counts().head(10)
        print(f"\n📖 FORMATS PHYSIQUES :")
        for format_type, count in format_counts.items():
            if format_type and format_type != '':
                percentage = (count / len(df)) * 100
                print(f"   • {format_type:<20} : {count:,} livres ({percentage:.1f}%)")
    
    # Analyse des pages
    if 'pages' in df.columns:
        pages_data = pd.to_numeric(df['pages'], errors='coerce').dropna()
        if len(pages_data) > 0:
            print(f"\n📄 STATISTIQUES DES PAGES :")
            print(f"   • Moyenne : {pages_data.mean():.0f} pages")
            print(f"   • Médiane : {pages_data.median():.0f} pages")
            print(f"   • Min - Max : {pages_data.min():.0f} - {pages_data.max():.0f} pages")
            
            # Répartition par tranches
            print(f"   • Répartition :")
            print(f"     - Moins de 100 pages : {(pages_data < 100).sum():,} livres")
            print(f"     - 100-300 pages : {((pages_data >= 100) & (pages_data <= 300)).sum():,} livres")
            print(f"     - Plus de 300 pages : {(pages_data > 300).sum():,} livres")
    
    # Taux de remplissage des métadonnées
    print(f"\n📋 QUALITÉ DES MÉTADONNÉES :")
    quality_fields = ['isbn_10', 'isbn_13', 'publishers', 'authors', 'subjects', 'description', 'covers']
    for field in quality_fields:
        if field in df.columns:
            non_empty = df[field].notna() & (df[field] != '')
            percentage = (non_empty.sum() / len(df)) * 100
            print(f"   • {field:<15} : {non_empty.sum():>6,} ({percentage:>5.1f}%)")
    
    # Exemples de livres de qualité
    print(f"\n📚 EXEMPLES DE LIVRES AVEC MÉTADONNÉES COMPLÈTES :")
    print("-" * 60)
    
    # Filtrer les livres avec beaucoup de métadonnées
    quality_score = 0
    for field in ['isbn_13', 'publishers', 'authors', 'subjects', 'description']:
        if field in df.columns:
            quality_score += (df[field].notna() & (df[field] != '')).astype(int)
    
    df['quality_score'] = quality_score
    high_quality = df[df['quality_score'] >= 4].head(5)
    
    for _, book in high_quality.iterrows():
        print(f"📖 {book['title']}")
        if pd.notna(book['subtitle']) and book['subtitle']:
            print(f"   Sous-titre: {book['subtitle']}")
        if pd.notna(book['publish_year']):
            print(f"   Année: {book['publish_year']:.0f}")
        if pd.notna(book['publishers']) and book['publishers']:
            publishers = book['publishers'][:100] + "..." if len(book['publishers']) > 100 else book['publishers']
            print(f"   Éditeur: {publishers}")
        if pd.notna(book['languages']) and book['languages']:
            print(f"   Langues: {book['languages']}")
        if pd.notna(book['pages']) and book['pages']:
            print(f"   Pages: {book['pages']}")
        print()
    
    return df

# LANCEMENT DE L'EXTRACTION SPÉCIALISÉE
if 'book_files' in globals() and book_files:
    editions_files = [f for f in book_files if f['type'] == 'editions']
    
    if editions_files:
        print(f"📚 EXTRACTION SPÉCIALISÉE D'ÉDITIONS")
        selected_file = editions_files[0]
        print(f"   Fichier : {selected_file['filename']}")
        
        # Définir les critères d'extraction
        extraction_criteria = {
            'has_isbn': True,           # Livres avec ISBN
            'has_authors': True,        # Livres avec auteurs
            'min_year': 1950,          # Depuis 1950
            'max_year': 2024,          # Jusqu'à 2024
            'languages': ['eng', 'fre', 'spa', 'ger', 'ita', 'por', 'rus', 'jpn', 'chi']  # Langues principales
        }
        
        # Extraire les livres
        specialized_books = extract_books_with_criteria(
            selected_file['path'], 
            max_books=15000, 
            criteria=extraction_criteria
        )
        
        if specialized_books:
            # Analyser les livres extraits
            df_specialized_books = analyze_extracted_books(specialized_books)
            print(f"\n💾 DataFrame 'df_specialized_books' créé avec {len(specialized_books):,} livres")
            print(f"\n📁 Pour sauvegarder en CSV :")
            print(f"   df_specialized_books.to_csv('livres_specialises.csv', index=False)")
        
else:
    print("❌ Aucun fichier d'éditions disponible pour l'extraction spécialisée")


🎯 EXTRACTION SPÉCIALISÉE DES LIVRES
📚 EXTRACTION SPÉCIALISÉE D'ÉDITIONS
   Fichier : autrices-auteurs-collections-maisons-edition-2018-05-01.csv
🔍 EXTRACTION AVEC CRITÈRES :
   • ISBN requis : True
   • Auteurs requis : True
   • Années : 1950 - 2024
   • Langues : eng, fre, spa, ger, ita, por, rus, jpn, chi
   • Maximum : 15,000 livres
--------------------------------------------------
✅ Extraction terminée : 0 livres trouvés sur 0 éditions traitées


In [29]:
# DIAGNOSTIC DES FICHIERS DE LIVRES
print("🔍 DIAGNOSTIC DES FICHIERS DE LIVRES")
print("=" * 60)

import os
import gzip

# Vérifier les fichiers disponibles
base_path = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary"

print("📁 FICHIERS DISPONIBLES :")
print("-" * 30)

if os.path.exists(base_path):
    files = os.listdir(base_path)
    for filename in files:
        file_path = os.path.join(base_path, filename)
        if os.path.isfile(file_path):
            size_mb = os.path.getsize(file_path) / (1024 * 1024)
            print(f"📄 {filename}")
            print(f"   Taille: {size_mb:.1f} MB")
            print()
else:
    print("❌ Le répertoire n'existe pas")

print("\n🔍 ANALYSE DÉTAILLÉE DES FICHIERS MIXTES :")
print("-" * 50)

# Analyser le fichier mixte principal
fichier_mixte = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_cdump_2025-05-31.txt.gz"

if os.path.exists(fichier_mixte):
    print(f"✅ Fichier mixte trouvé : {os.path.basename(fichier_mixte)}")
    
    # Analyser les 1000 premières lignes
    print("\n📊 ANALYSE DES TYPES D'ENTRÉES (1000 premières lignes) :")
    
    type_counts = {}
    sample_entries = {}
    
    try:
        with gzip.open(fichier_mixte, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 1000:
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 1:
                    entry_type = parts[0]
                    type_counts[entry_type] = type_counts.get(entry_type, 0) + 1
                    
                    # Garder un échantillon de chaque type
                    if entry_type not in sample_entries:
                        sample_entries[entry_type] = {
                            'line': line.strip(),
                            'parts_count': len(parts)
                        }
        
        print(f"Types d'entrées trouvés :")
        for entry_type, count in sorted(type_counts.items()):
            print(f"   • {entry_type:<20} : {count:,} entrées")
        
        print(f"\n📋 ÉCHANTILLONS DE CHAQUE TYPE :")
        for entry_type, sample in sample_entries.items():
            print(f"\n🔸 {entry_type} ({sample['parts_count']} parties) :")
            # Afficher seulement les premières parties pour éviter le JSON complet
            parts = sample['line'].split('\t')[:4]
            for i, part in enumerate(parts):
                print(f"   [{i}] {part[:100]}{'...' if len(part) > 100 else ''}")
                
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")

else:
    print("❌ Fichier mixte non trouvé")

print("\n🔍 ANALYSE SPÉCIFIQUE DES ÉDITIONS :")
print("-" * 40)

# Rechercher spécifiquement les éditions dans le fichier mixte
if os.path.exists(fichier_mixte):
    editions_found = 0
    works_found = 0
    authors_found = 0
    
    try:
        with gzip.open(fichier_mixte, 'rt', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= 5000:  # Analyser plus de lignes
                    break
                
                parts = line.strip().split('\t')
                if len(parts) >= 5:
                    entry_type = parts[0]
                    
                    if entry_type == '/type/edition':
                        editions_found += 1
                        if editions_found == 1:  # Premier exemple d'édition
                            print(f"📚 PREMIÈRE ÉDITION TROUVÉE (ligne {i+1}) :")
                            try:
                                import json
                                json_data = json.loads(parts[4])
                                print(f"   • ID: {parts[1]}")
                                print(f"   • Titre: {json_data.get('title', 'N/A')}")
                                print(f"   • Date: {json_data.get('publish_date', 'N/A')}")
                                print(f"   • ISBN: {json_data.get('isbn_13', json_data.get('isbn_10', 'N/A'))}")
                                print(f"   • Auteurs: {'Oui' if 'authors' in json_data else 'Non'}")
                            except:
                                print("   • Erreur lors du parsing JSON")
                    
                    elif entry_type == '/type/work':
                        works_found += 1
                    elif entry_type == '/type/author':
                        authors_found += 1
        
        print(f"\n📊 RÉSULTATS DE L'ANALYSE (5000 premières lignes) :")
        print(f"   • Éditions trouvées : {editions_found:,}")
        print(f"   • Œuvres trouvées : {works_found:,}")
        print(f"   • Auteurs trouvés : {authors_found:,}")
        
        if editions_found == 0:
            print(f"\n⚠️ AUCUNE ÉDITION TROUVÉE dans les 5000 premières lignes !")
            print(f"   Le fichier pourrait contenir les éditions plus loin.")
            print(f"   Essayons de chercher plus loin...")
            
            # Recherche rapide dans tout le fichier
            print(f"\n🔍 RECHERCHE RAPIDE DES ÉDITIONS :")
            try:
                with gzip.open(fichier_mixte, 'rt', encoding='utf-8') as f:
                    for i, line in enumerate(f):
                        if i % 10000 == 0:  # Échantillonner chaque 10000e ligne
                            parts = line.strip().split('\t')
                            if len(parts) >= 1 and parts[0] == '/type/edition':
                                print(f"   ✅ Première édition trouvée à la ligne ~{i+1}")
                                break
                        if i > 100000:  # Limiter la recherche
                            break
                    else:
                        print(f"   ❌ Aucune édition trouvée dans les 100,000 premières lignes")
            except Exception as e:
                print(f"   ❌ Erreur lors de la recherche : {e}")
        
    except Exception as e:
        print(f"❌ Erreur lors de l'analyse : {e}")

print(f"\n💡 SUGGESTIONS :")
print(f"   1. Le fichier mixte contient peut-être les éditions plus loin")
print(f"   2. Il pourrait y avoir un fichier d'éditions séparé")
print(f"   3. La structure du fichier pourrait être différente")


🔍 DIAGNOSTIC DES FICHIERS DE LIVRES
📁 FICHIERS DISPONIBLES :
------------------------------
📄 autrices-auteurs-collections-maisons-edition-2018-05-01.csv
   Taille: 0.2 MB

📄 extracted_authors.csv
   Taille: 740.2 MB

📄 isbn-open-data.zip
   Taille: 436.5 MB

📄 ol_cdump_2025-05-31.txt.gz.opdownload
   Taille: 40333.3 MB

📄 ol_dump_authors_2025-05-31.txt.gz
   Taille: 678.1 MB

📄 ol_dump_editions_2025-05-31.txt.gz
   Taille: 10969.8 MB

📄 ol_dump_lists_2025-05-31.txt.gz
   Taille: 35.3 MB

📄 test_authors_100k.csv
   Taille: 5.1 MB


🔍 ANALYSE DÉTAILLÉE DES FICHIERS MIXTES :
--------------------------------------------------
❌ Fichier mixte non trouvé

🔍 ANALYSE SPÉCIFIQUE DES ÉDITIONS :
----------------------------------------

💡 SUGGESTIONS :
   1. Le fichier mixte contient peut-être les éditions plus loin
   2. Il pourrait y avoir un fichier d'éditions séparé
   3. La structure du fichier pourrait être différente


In [ ]:
# 🔄 MISE À JOUR DE LA BASE DE DONNÉES AVEC LES DATES DE MORT
print("🔄 MISE À JOUR DE LA BASE DE DONNÉES")
print("=" * 50)

# Connexion à la base de données (ajustez selon votre configuration)
import sqlite3  # ou utilisez votre connecteur de base de données

def connect_to_database():
    """Connexion à votre base de données"""
    # AJUSTEZ CES PARAMÈTRES SELON VOTRE BASE DE DONNÉES
    
    # Pour SQLite (exemple)
    # return sqlite3.connect('path/to/your/database.db')
    
    # Pour MySQL/MariaDB
    # import mysql.connector
    # return mysql.connector.connect(
    #     host='localhost',
    #     database='databook',
    #     user='your_username',
    #     password='your_password'
    # )
    
    # Pour PostgreSQL
    # import psycopg2
    # return psycopg2.connect(
    #     host='localhost',
    #     database='databook',
    #     user='your_username',
    #     password='your_password'
    # )
    
    print("⚠️ Veuillez configurer la connexion à votre base de données")
    return None

def add_death_date_column():
    """Ajouter la colonne date_mort si elle n'existe pas"""
    try:
        conn = connect_to_database()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        # Vérifier si la colonne existe déjà
        # La requête varie selon le type de base de données
        
        # Pour MySQL/MariaDB
        check_column_sql = """
        SELECT COUNT(*) 
        FROM INFORMATION_SCHEMA.COLUMNS 
        WHERE TABLE_NAME = 'auteur' 
          AND COLUMN_NAME = 'date_mort'
          AND TABLE_SCHEMA = DATABASE()
        """
        
        # Pour SQLite
        # check_column_sql = "PRAGMA table_info(auteur)"
        
        cursor.execute(check_column_sql)
        column_exists = cursor.fetchone()[0] > 0
        
        if not column_exists:
            print("📝 Ajout de la colonne date_mort...")
            
            # Ajouter la colonne
            alter_sql = """
            ALTER TABLE auteur 
            ADD COLUMN date_mort VARCHAR(100) NULL 
            """
            
            cursor.execute(alter_sql)
            conn.commit()
            print("✅ Colonne date_mort ajoutée avec succès")
        else:
            print("ℹ️ La colonne date_mort existe déjà")
        
        conn.close()
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de l'ajout de la colonne : {e}")
        return False

def update_death_dates_from_dataframe():
    """Mettre à jour les dates de mort depuis le DataFrame des auteurs"""
    
    # Vérifier si nous avons des données d'auteurs avec dates de naissance
    if 'df_authors_birth' not in globals():
        print("❌ DataFrame 'df_authors_birth' non trouvé")
        print("   Veuillez d'abord exécuter l'extraction des auteurs (cellules 16-18)")
        return False
    
    try:
        conn = connect_to_database()
        if not conn:
            return False
        
        cursor = conn.cursor()
        
        # Filtrer les auteurs avec date de mort
        authors_with_death = df_authors_birth[
            (df_authors_birth['death_date'].notna()) & 
            (df_authors_birth['death_date'] != '')
        ].copy()
        
        print(f"🔍 Trouvé {len(authors_with_death):,} auteurs avec date de mort")
        
        if len(authors_with_death) == 0:
            print("ℹ️ Aucun auteur avec date de mort à mettre à jour")
            return True
        
        # Préparer les données pour la mise à jour
        updates_made = 0
        
        for _, author in authors_with_death.iterrows():
            author_id = author['author_id']
            death_date = author['death_date']
            
            # Chercher l'auteur dans la base par ID OpenLibrary
            find_sql = "SELECT id_auteur FROM auteur WHERE id_auteur LIKE %s"
            cursor.execute(find_sql, (f"%{author_id}%",))
            result = cursor.fetchone()
            
            if result:
                # Mettre à jour la date de mort
                update_sql = """
                UPDATE auteur 
                SET date_mort = %s 
                WHERE id_auteur LIKE %s
                """
                cursor.execute(update_sql, (death_date, f"%{author_id}%"))
                updates_made += 1
        
        conn.commit()
        print(f"✅ Mise à jour terminée : {updates_made:,} auteurs mis à jour")
        
        conn.close()
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de la mise à jour : {e}")
        return False

def create_author_mapping():
    """Créer un mapping entre les auteurs OpenLibrary et la base locale"""
    
    if 'df_authors_birth' not in globals():
        print("❌ DataFrame 'df_authors_birth' non trouvé")
        return None
    
    try:
        conn = connect_to_database()
        if not conn:
            return None
        
        cursor = conn.cursor()
        
        # Récupérer tous les auteurs de la base locale
        cursor.execute("SELECT id_auteur, nom FROM auteur")
        local_authors = cursor.fetchall()
        
        print(f"📊 Auteurs dans la base locale : {len(local_authors):,}")
        print(f"📊 Auteurs OpenLibrary avec dates : {len(df_authors_birth):,}")
        
        # Créer un DataFrame de mapping
        mapping_data = []
        
        for local_id, local_name in local_authors:
            # Chercher des correspondances par nom
            matches = df_authors_birth[
                df_authors_birth['name'].str.contains(local_name, case=False, na=False)
            ]
            
            if len(matches) > 0:
                best_match = matches.iloc[0]  # Prendre la première correspondance
                mapping_data.append({
                    'local_id': local_id,
                    'local_name': local_name,
                    'openlibrary_id': best_match['author_id'],
                    'openlibrary_name': best_match['name'],
                    'birth_date': best_match['birth_date'],
                    'death_date': best_match['death_date'],
                    'birth_year': best_match['birth_year'],
                    'death_year': best_match['death_year']
                })
        
        conn.close()
        
        if mapping_data:
            mapping_df = pd.DataFrame(mapping_data)
            print(f"✅ Correspondances trouvées : {len(mapping_df):,}")
            return mapping_df
        else:
            print("❌ Aucune correspondance trouvée")
            return None
            
    except Exception as e:
        print(f"❌ Erreur lors du mapping : {e}")
        return None

# EXÉCUTION DES OPÉRATIONS
print("🚀 DÉMARRAGE DES OPÉRATIONS DE BASE DE DONNÉES")
print()

print("1️⃣ CONFIGURATION DE LA CONNEXION :")
print("   ⚠️  IMPORTANT : Configurez d'abord la fonction connect_to_database()")
print("   📝 Modifiez les paramètres de connexion selon votre base de données")
print()

print("2️⃣ OPÉRATIONS DISPONIBLES :")
print("   • add_death_date_column() - Ajouter la colonne date_mort")
print("   • update_death_dates_from_dataframe() - Mettre à jour depuis OpenLibrary")
print("   • create_author_mapping() - Créer un mapping des correspondances")
print()

print("💡 EXEMPLE D'UTILISATION :")
print("   # 1. Ajouter la colonne")
print("   # add_death_date_column()")
print("   # ")
print("   # 2. Créer le mapping")
print("   # mapping_df = create_author_mapping()")
print("   # ")
print("   # 3. Mettre à jour les dates")
print("   # update_death_dates_from_dataframe()")

print("\n⚠️ ATTENTION : Configurez d'abord votre connexion à la base de données !")


In [30]:
# EXTRACTION DIRECTE DES ÉDITIONS
print("📚 EXTRACTION DIRECTE DES ÉDITIONS")
print("=" * 60)

# Utiliser le fichier d'éditions spécifique
fichier_editions = r"C:\Users\dd758\Formation_IA_Greta\Projet_possible certif\Livre_analyse\data_book\databook\data\fichier_openlibrary\ol_dump_editions_2025-05-31.txt.gz"

def extract_editions_simple(filename, max_books=5000):
    """Extraction simple et directe des éditions"""
    print(f"🔍 EXTRACTION DEPUIS : {os.path.basename(filename)}")
    print(f"   Limite : {max_books:,} éditions")
    print("-" * 50)
    
    books_data = []
    total_lines = 0
    editions_processed = 0
    books_extracted = 0
    
    try:
        # Déterminer si le fichier est compressé
        open_func = gzip.open if filename.endswith('.gz') else open
        mode = 'rt' if filename.endswith('.gz') else 'r'
        
        with open_func(filename, mode, encoding='utf-8') as f:
            for line_num, line in enumerate(f):
                total_lines += 1
                
                if books_extracted >= max_books:
                    break
                
                # Traiter chaque ligne
                parts = line.strip().split('\t')
                
                # Vérifier la structure de la ligne
                if len(parts) >= 5:
                    entry_type = parts[0]
                    
                    # Ne traiter que les éditions
                    if entry_type == '/type/edition':
                        editions_processed += 1
                        
                        try:
                            # Parser le JSON
                            json_data = json.loads(parts[4])
                            
                            # Extraire les informations de base
                            book_info = {
                                'book_id': parts[1],
                                'revision': parts[2],
                                'created': parts[3],
                                'title': json_data.get('title', ''),
                                'subtitle': json_data.get('subtitle', ''),
                                'publish_date': json_data.get('publish_date', ''),
                                'publish_year': None,
                                'isbn_10': '',
                                'isbn_13': '',
                                'publishers': '',
                                'languages': '',
                                'authors': '',
                                'pages': json_data.get('number_of_pages', ''),
                                'physical_format': json_data.get('physical_format', ''),
                                'edition_name': json_data.get('edition_name', ''),
                            }
                            
                            # Extraire l'année de publication
                            if book_info['publish_date']:
                                year_match = re.search(r'\b(\d{4})\b', str(book_info['publish_date']))
                                if year_match:
                                    year = int(year_match.group(1))
                                    if 1000 <= year <= datetime.now().year:
                                        book_info['publish_year'] = year
                            
                            # Extraire ISBN
                            if 'isbn_10' in json_data and json_data['isbn_10']:
                                if isinstance(json_data['isbn_10'], list):
                                    book_info['isbn_10'] = json_data['isbn_10'][0]  # Prendre le premier
                                else:
                                    book_info['isbn_10'] = str(json_data['isbn_10'])
                            
                            if 'isbn_13' in json_data and json_data['isbn_13']:
                                if isinstance(json_data['isbn_13'], list):
                                    book_info['isbn_13'] = json_data['isbn_13'][0]  # Prendre le premier
                                else:
                                    book_info['isbn_13'] = str(json_data['isbn_13'])
                            
                            # Extraire éditeurs
                            if 'publishers' in json_data and json_data['publishers']:
                                if isinstance(json_data['publishers'], list):
                                    book_info['publishers'] = json_data['publishers'][0]  # Prendre le premier
                                else:
                                    book_info['publishers'] = str(json_data['publishers'])
                            
                            # Extraire langues
                            if 'languages' in json_data and json_data['languages']:
                                languages = []
                                for lang in json_data['languages']:
                                    if isinstance(lang, dict) and 'key' in lang:
                                        lang_code = lang['key'].replace('/languages/', '')
                                        languages.append(lang_code)
                                    else:
                                        languages.append(str(lang))
                                book_info['languages'] = languages[0] if languages else ''  # Prendre la première
                            
                            # Extraire auteurs (IDs seulement pour simplicité)
                            if 'authors' in json_data and json_data['authors']:
                                authors = []
                                for author in json_data['authors']:
                                    if isinstance(author, dict) and 'key' in author:
                                        authors.append(author['key'])
                                    else:
                                        authors.append(str(author))
                                book_info['authors'] = authors[0] if authors else ''  # Prendre le premier
                            
                            # Ajouter le livre seulement s'il a un titre
                            if book_info['title']:
                                books_data.append(book_info)
                                books_extracted += 1
                        
                        except json.JSONDecodeError as e:
                            # Ignorer les erreurs JSON
                            continue
                        except Exception as e:
                            # Ignorer les autres erreurs
                            continue
                
                # Afficher le progrès
                if total_lines % 10000 == 0:
                    print(f"   Ligne {total_lines:,} - Éditions: {editions_processed:,} - Extraits: {books_extracted:,}")
    
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction : {e}")
        return None
    
    print(f"\n✅ EXTRACTION TERMINÉE :")
    print(f"   • Total lignes lues : {total_lines:,}")
    print(f"   • Éditions trouvées : {editions_processed:,}")
    print(f"   • Livres extraits : {books_extracted:,}")
    
    return books_data

def analyze_simple_books(books_data):
    """Analyse simple des livres extraits"""
    if not books_data:
        print("❌ Aucune donnée à analyser")
        return None
    
    print(f"\n📊 ANALYSE DES LIVRES EXTRAITS")
    print("=" * 50)
    
    df = pd.DataFrame(books_data)
    print(f"📏 DataFrame créé : {df.shape[0]:,} livres × {df.shape[1]} colonnes")
    
    # Statistiques de base
    print(f"\n📋 MÉTADONNÉES DISPONIBLES :")
    for col in ['title', 'publish_year', 'isbn_13', 'isbn_10', 'publishers', 'languages', 'authors', 'pages']:
        if col in df.columns:
            non_empty = df[col].notna() & (df[col] != '')
            percentage = (non_empty.sum() / len(df)) * 100
            print(f"   • {col:<15} : {non_empty.sum():>5,} ({percentage:>5.1f}%)")
    
    # Années de publication
    if 'publish_year' in df.columns:
        years = df['publish_year'].dropna()
        if len(years) > 0:
            print(f"\n📅 ANNÉES DE PUBLICATION :")
            print(f"   • Range : {years.min():.0f} - {years.max():.0f}")
            print(f"   • Moyenne : {years.mean():.0f}")
            
            # Quelques décennies récentes
            for decade in [1980, 1990, 2000, 2010, 2020]:
                count = years[(years >= decade) & (years < decade + 10)].count()
                if count > 0:
                    print(f"   • {decade}s : {count:,} livres")
    
    # Langues
    if 'languages' in df.columns:
        lang_counts = df['languages'].value_counts().head(10)
        print(f"\n🌍 LANGUES LES PLUS FRÉQUENTES :")
        for lang, count in lang_counts.items():
            if lang and lang != '':
                print(f"   • {lang:<10} : {count:,} livres")
    
    # Quelques exemples
    print(f"\n📚 EXEMPLES DE LIVRES :")
    print("-" * 40)
    
    sample_books = df.head(5)
    for i, (_, book) in enumerate(sample_books.iterrows(), 1):
        print(f"{i}. 📖 {book['title']}")
        if pd.notna(book['publish_year']):
            print(f"   Année : {book['publish_year']:.0f}")
        if pd.notna(book['publishers']) and book['publishers']:
            print(f"   Éditeur : {book['publishers'][:50]}...")
        if pd.notna(book['languages']) and book['languages']:
            print(f"   Langue : {book['languages']}")
        if pd.notna(book['isbn_13']) and book['isbn_13']:
            print(f"   ISBN-13 : {book['isbn_13']}")
        print()
    
    return df

# EXÉCUTION DE L'EXTRACTION
if os.path.exists(fichier_editions):
    print(f"✅ Fichier d'éditions trouvé")
    
    # Extraire les livres
    books_data = extract_editions_simple(fichier_editions, max_books=10000)
    
    if books_data and len(books_data) > 0:
        # Analyser les résultats
        df_livres = analyze_simple_books(books_data)
        
        print(f"\n💾 RÉSULTAT :")
        print(f"   DataFrame 'df_livres' créé avec {len(books_data):,} livres")
        print(f"\n📁 SAUVEGARDE :")
        print(f"   df_livres.to_csv('livres_openlibrary.csv', index=False)")
    else:
        print("❌ Aucun livre extrait")

else:
    print(f"❌ Fichier d'éditions non trouvé : {fichier_editions}")
    print(f"\n📁 Fichiers disponibles dans le répertoire :")
    if os.path.exists(os.path.dirname(fichier_editions)):
        for f in os.listdir(os.path.dirname(fichier_editions)):
            print(f"   • {f}")


📚 EXTRACTION DIRECTE DES ÉDITIONS
✅ Fichier d'éditions trouvé
🔍 EXTRACTION DEPUIS : ol_dump_editions_2025-05-31.txt.gz
   Limite : 10,000 éditions
--------------------------------------------------
   Ligne 10,000 - Éditions: 10,000 - Extraits: 10,000

✅ EXTRACTION TERMINÉE :
   • Total lignes lues : 10,001
   • Éditions trouvées : 10,000
   • Livres extraits : 10,000

📊 ANALYSE DES LIVRES EXTRAITS
📏 DataFrame créé : 10,000 livres × 15 colonnes

📋 MÉTADONNÉES DISPONIBLES :
   • title           : 10,000 (100.0%)
   • publish_year    : 9,684 ( 96.8%)
   • isbn_13         : 8,871 ( 88.7%)
   • isbn_10         : 9,703 ( 97.0%)
   • publishers      : 9,894 ( 98.9%)
   • languages       : 6,608 ( 66.1%)
   • authors         : 8,253 ( 82.5%)
   • pages           : 6,710 ( 67.1%)

📅 ANNÉES DE PUBLICATION :
   • Range : 1657 - 2014
   • Moyenne : 1995
   • 1980s : 1,251 livres
   • 1990s : 3,914 livres
   • 2000s : 3,899 livres
   • 2010s : 3 livres

🌍 LANGUES LES PLUS FRÉQUENTES :
   • eng    

### Formatage objet livre pour pouvoir les insérer dans la base de données sql